In [1]:
import re
import numpy as np 
import folium

In [2]:
#re?
#re.search?
#re.match?
#re.sub?
#re.findall?
#folium?

In [3]:
#
# Using the numpy array can speed up your code a lot !!! Check this version.

def get_osm ( url ):
#
#   Usage: 
#   url = 'http://115.78.93.252/TTU/TpHCM.osm'
#   node_ref, node_ref1, way_ref, rel_ref, node_lon, node_lat, node_info, way_info, rel_info = get_osm ( url )
#
    import re
    from urllib.request import urlopen
#
#
    def trim_l ( line ):
        return re.sub( r'^\s+', '', line)
    def trim_r ( line ):
        return re.sub( r'\s+$', '', line)
    def trim ( line ):
        return trim_l( trim_r(line) )
    def is_node_s ( a ):
        return re.match ( r'^.*<node[\s+,>].*$', a, flags=re.IGNORECASE ) is not None 
    def is_node_e ( a ):
        return re.match ( r'^.*</node>.*$', a, flags=re.IGNORECASE ) is not None  
    def is_elem_s ( tag, a, flags=re.IGNORECASE ):
        return re.match ( r'^.*<'+str(tag)+'[\s+,>].*$', a, flags=flags ) is not None 
    def is_elem_e ( tag, a, flags=re.IGNORECASE ):
        return re.match ( r'^.*</'+str(tag)+'>.*$', a, flags=flags ) is not None 
    def getnode_id_lon_lat ( line ):
        xid  = re.sub( r'^.*\s+id="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line )
        xlon = re.sub( r'^.*\s+lon="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line )
        xlat = re.sub( r'^.*\s+lat="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line )
        return int(xid), float(xlat), float(xlon) 
    def get_id_lon_lat ( line ):
        if ( is_node_s (line) ):
            return getnode_id_lon_lat ( line )
        else:
            return None, None, None 
    def getelem_id ( line ):
        xid  = re.sub( r'^.*\s+id="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', line )
        return int(xid)
    def gettag_int ( tag, line ):
        xid  = re.sub( r'^.*\s+'+str(tag)+'="([-+]?[0-9]*)"\s+.*$', r'\1', line )
        return int(xid)
#
#
    Text = re.split( '\\n+', urlopen(url).read().decode()  )    
#    
#
#   1. To extract "node" information (id, lat, lon) and data (id_ref, tags):
#

    node_ref = []
    node_lon = []
    node_lat = []
    
    node_ref1 = []
    node_info = []

    nl = 0
    flag_s = flag_e = False 
    xid = xlat = xlon = 0
    xdata = ''
    flag_open = False

    for line in Text:
        nl = nl + 1 
        flag_s = is_node_s (line)
        flag_e = is_node_e (line)
        if ( flag_s ):
            flag_open = True
            xid, xlat, xlon = getnode_id_lon_lat ( line )
            node_ref.append ( xid )
            node_lat.append ( xlat )
            node_lon.append ( xlon )
            xdata = ''
        elif ( flag_e ):
            flag_open = False
            node_ref1.append  ( xid )
            node_info.append ( xdata )
        else:
            if ( flag_open ): 
                xdata = xdata + str(trim(line))
#
    print ( "%4i %40s %15d" % ( 1, 'Reading Nodes over No. lines', nl) )    
    print ( "%45s %15d" % ( 'No. Nodes =', len(node_ref) ) )    
    
#
#   2. To extract "way" information (id) and way data (node_ref, tags, etc)
#      The "way" data is a HUGE data set !!!

    way_ref = []
    way_info = []

    nl = 0
    flag_s = flag_e = False 
    wid = 0
    wdata = ''
    flag_open = False

    for line in Text:
        nl = nl + 1 
        flag_s = is_elem_s ( 'way', line )
        flag_e = is_elem_e ( 'way', line )
        if ( flag_s ):
            flag_open = True
            wid = getelem_id ( line )
            way_ref.append  ( wid )
            wdata = ''
        elif ( flag_e ):
            flag_open = False
            way_info.append ( wdata )
        else:
            if ( flag_open ): 
                wdata = wdata + str(trim(line))
#
    print ( "%4i %40s %15d" % ( 2, 'Reading Way over No. lines', nl) )    
    print ( "%45s %15d" % ('No. Way =', len(way_ref)) )    
    
#
#   3. To extract "relation" information (id) and data (ref, tags, etc)
#
    rel_ref = []
    rel_info = []

    nl = 0
    flag_s = flag_e = False 
    rid = 0
    rdata = ''
    flag_open = False

    for line in Text:
        nl = nl + 1 
        flag_s = is_elem_s ( 'relation', line )
        flag_e = is_elem_e ( 'relation', line )
        if ( flag_s ):
            flag_open = True
            rid = getelem_id ( line )
            rel_ref.append ( rid )
            rdata = ''
        elif ( flag_e ):
            flag_open = False
            rel_info.append ( rdata )
        else:
            if ( flag_open ): 
                rdata = rdata + str(trim(line))
#
    print ( "%4i %40s %15d" % ( 3, 'Reading Relation over No. lines', nl) )    
    print ( "%45s %15d" % ('No. Relation =', len(rel_ref)) )    
#    
    return  \
    np.dtype('int64').type( node_ref ), \
    np.dtype('int64').type( node_ref1 ), \
    np.dtype('int64').type( way_ref ), \
    np.dtype('int64').type( rel_ref ), \
    np.dtype('float64').type( node_lon ), \
    np.dtype('float64').type( node_lat ), \
    node_info, \
    way_info, \
    rel_info
#

<>:25: SyntaxWarning: invalid escape sequence '\s'
<>:42: SyntaxWarning: invalid escape sequence '\s'
<>:25: SyntaxWarning: invalid escape sequence '\s'
<>:42: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_95462/3917678736.py:25: SyntaxWarning: invalid escape sequence '\s'
  return re.match ( r'^.*<'+str(tag)+'[\s+,>].*$', a, flags=flags ) is not None
/tmp/ipykernel_95462/3917678736.py:42: SyntaxWarning: invalid escape sequence '\s'
  xid  = re.sub( r'^.*\s+'+str(tag)+'="([-+]?[0-9]*)"\s+.*$', r'\1', line )


In [4]:
url = 'http://115.78.93.252/TTU/TpHCM.osm'

node_ref, node_ref1, way_ref, rel_ref, node_lon, node_lat, node_info, way_info, rel_info = get_osm ( url )

   1             Reading Nodes over No. lines         7686296
                                  No. Nodes =         2558903
   2               Reading Way over No. lines         7686296
                                    No. Way =          419392
   3          Reading Relation over No. lines         7686296
                               No. Relation =            3411


In [5]:
node_ref.dtype, node_ref1.dtype, way_ref.dtype, rel_ref.dtype, node_lon.dtype, node_lat.dtype

(dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'))


# Check somewhat it have read

In [6]:
def trim_l ( line ):
    return re.sub( r'^\s+', '', line)
def trim_r ( line ):
    return re.sub( r'\s+$', '', line)
def trim ( line ):
    return trim_l( trim_r(line) )

In [7]:
n = 3

print ( "\n*** Nodes:" )
print ( ' %10s %20s %22s %22s' % ( 'index (id)', 'reference (ref)', 'longitude (lon)', 'latitude (lat)' ) )
for k in range(n):
    print ( ' %10d %20d %22.12f %22.12f' % ( k, node_ref[k], node_lon[k], node_lat[k] ) )
#    

print ( "\n*** Node data:" )
print ( ' %10s %20s %50s' % ( 'index (id)', 'reference (ref)', 'information (info)' ) )
for k in range(n):
    print ( ' %10d %20d %50s' % ( k, node_ref1[k], node_info[k] ) )    
#

print ( "\n*** Way data:" )
print ( ' %10s %20s %50s' % ( 'index (id)', 'reference (ref)', 'information (info)' ) )
for k in range(n):
    print ( ' %10d %20d %50s' % ( k, way_ref[k], way_info[k] ) )  
#


print ( "\n*** Relation data:" )
print ( ' %10s %20s %50s' % ( 'index (id)', 'reference (ref)', 'information (info)' ) )
for k in range(n):
    print ( ' %10d %20d %50s' % ( k, rel_ref[k], rel_info[k] ) )  
#




*** Nodes:
 index (id)      reference (ref)        longitude (lon)         latitude (lat)
          0             91665273       106.891890600000        10.970310800000
          1             91665277       106.871587400000        10.951023600000
          2             91665279       106.868371400000        10.943379800000

*** Node data:
 index (id)      reference (ref)                                 information (info)
          0            305327873                      <tag k="railway" v="switch"/>
          1            305328007 <tag k="crossing:barrier" v="full"/><tag k="crossing:bell" v="yes"/><tag k="crossing:light" v="yes"/><tag k="railway" v="level_crossing"/>
          2            305328010                      <tag k="railway" v="switch"/>

*** Way data:
 index (id)      reference (ref)                                 information (info)
          0             10578009 <nd ref="91778675"/><nd ref="1608787098"/><nd ref="1608787169"/><nd ref="1608787415"/><nd ref="16087

In [8]:
j = 3
rel_info [j]

'<member type="way" ref="32661173" role="inner"/><member type="way" ref="41241468" role="outer"/><tag k="name" v="Rạch Long Vương"/><tag k="natural" v="water"/><tag k="type" v="multipolygon"/><tag k="water" v="river"/>'

In [9]:
n_tags = 0 
k_hcm = 0 

for k in range( len(rel_info) ):
    line = rel_info[k]
    if ( re.match( r'^.*(HCM).*$', line, flags=re.IGNORECASE ) is not None ) :
        if ( re.match( r'^.*(<tag k="name" v="Thành phố Hồ Chí Minh).*$', line, flags=re.IGNORECASE ) is not None ) :
            n_tags = n_tags + 1 
            k_hcm = k 
            print ( k, rel_ref[k], line, '\n' )
#
print ( 'No. tags =', n_tags, '\n' )            

31 1973756 <member type="node" ref="369486734" role="admin_centre"/><member type="relation" ref="2587287" role="subarea"/><member type="relation" ref="3819816" role="subarea"/><member type="relation" ref="2778323" role="subarea"/><member type="relation" ref="3820432" role="subarea"/><member type="relation" ref="6228792" role="subarea"/><member type="relation" ref="2764875" role="subarea"/><member type="relation" ref="6888445" role="subarea"/><member type="relation" ref="6228121" role="subarea"/><member type="relation" ref="6846181" role="subarea"/><member type="relation" ref="6923167" role="subarea"/><member type="relation" ref="7157268" role="subarea"/><member type="relation" ref="3797166" role="subarea"/><member type="relation" ref="7157255" role="subarea"/><member type="relation" ref="7157220" role="subarea"/><member type="relation" ref="7157219" role="subarea"/><member type="relation" ref="7157197" role="subarea"/><member type="relation" ref="3851694" role="subarea"/><member type="

In [10]:
tphcm = rel_info [ k_hcm ]
tphcm 

'<member type="node" ref="369486734" role="admin_centre"/><member type="relation" ref="2587287" role="subarea"/><member type="relation" ref="3819816" role="subarea"/><member type="relation" ref="2778323" role="subarea"/><member type="relation" ref="3820432" role="subarea"/><member type="relation" ref="6228792" role="subarea"/><member type="relation" ref="2764875" role="subarea"/><member type="relation" ref="6888445" role="subarea"/><member type="relation" ref="6228121" role="subarea"/><member type="relation" ref="6846181" role="subarea"/><member type="relation" ref="6923167" role="subarea"/><member type="relation" ref="7157268" role="subarea"/><member type="relation" ref="3797166" role="subarea"/><member type="relation" ref="7157255" role="subarea"/><member type="relation" ref="7157220" role="subarea"/><member type="relation" ref="7157219" role="subarea"/><member type="relation" ref="7157197" role="subarea"/><member type="relation" ref="3851694" role="subarea"/><member type="relation" 

# Note: 
+ ## References of node, way, relation are uniquely determined. 
+ ## Do not use indices to store the information since they are varied among arrays.

# The information may be huge. Therefore, we will use only reference instead of copying information to avoid overloading the RAM. In the circumstance, we should write a routine querying information quickly from a given reference: 
## For a list of information A, given a reference, we find index so that 
+ ## reference --> index --> information A [ index ]

## Here "A" may be a table and "index" is its row's index. Such indices are varied from table to table.

# Part 1. RELATION

# How to get information of Quận ? First use Relation !

# Quan = District

In [11]:
n_tags = 0 

for k in range( len(rel_info) ):
    line = rel_info[k]
    if ( re.match( r'^.*(Quận).*$', line, flags=re.IGNORECASE ) is not None ) :
        if ( re.match( r'^.*(<tag k="name" v="Quận\s+).*$', line, flags=re.IGNORECASE ) is not None ) :
            n_tags = n_tags + 1 
            print ( k, rel_ref[k], line, '\n' )
#
print ( 'No. Quan =', n_tags, '\n' )            

47 2587287 <member type="way" ref="287982010" role="outer"/><member type="way" ref="189393381" role="outer"/><member type="way" ref="189618477" role="outer"/><member type="way" ref="418298179" role="outer"/><member type="way" ref="418298182" role="outer"/><member type="way" ref="189233380" role="outer"/><member type="way" ref="192038542" role="outer"/><member type="way" ref="192680136" role="outer"/><member type="way" ref="189619670" role="outer"/><member type="way" ref="206677421" role="outer"/><member type="way" ref="472074786" role="outer"/><member type="way" ref="472074778" role="outer"/><member type="way" ref="191877594" role="outer"/><member type="way" ref="472236749" role="outer"/><member type="way" ref="191877592" role="outer"/><member type="way" ref="286342012" role="outer"/><member type="way" ref="888760606" role="outer"/><member type="way" ref="286341995" role="outer"/><member type="way" ref="191881069" role="outer"/><member type="way" ref="289931230" role="outer"/><member t

In [12]:
# Pick up one of them to try 

k = 379
line = rel_info[k]
line 

# Look at this: "<tag k="name" v="Quận 12"/>" !

'<member type="way" ref="470198970" role="outer"/><member type="way" ref="469350123" role="outer"/><member type="way" ref="468652907" role="outer"/><member type="way" ref="466933218" role="outer"/><member type="way" ref="467114640" role="outer"/><member type="way" ref="470205581" role="outer"/><member type="way" ref="468749926" role="outer"/><member type="way" ref="470205586" role="outer"/><member type="way" ref="471507104" role="outer"/><member type="way" ref="468749924" role="outer"/><member type="way" ref="471507115" role="outer"/><member type="way" ref="468749928" role="outer"/><member type="way" ref="468659640" role="outer"/><member type="way" ref="471507117" role="outer"/><member type="way" ref="468659638" role="outer"/><member type="way" ref="286462029" role="outer"/><member type="way" ref="470198974" role="outer"/><member type="way" ref="471507118" role="outer"/><member type="way" ref="1005422923" role="outer"/><member type="way" ref="470198972" role="outer"/><member type="way"

In [13]:
# How to extract that "Quận 12" from there ? 

qname  = re.sub( r'^.*(<tag k="name" v=")(Quận\s+[0-9]+)("/>.*)$', r'\2', line )
qname 

# Can you see this ?

'Quận 12'

In [14]:
k = 111
line = rel_info[k]
qname  = re.sub( r'^.*(<tag k="name" v=")(Quận\s+[0-9]+)("/>.*)$', r'\2', line )
qname 

# But, this gives a wrong answer ! 

'<member type="way" ref="286342044" role="outer"/><member type="way" ref="286330736" role="outer"/><member type="way" ref="286462029" role="outer"/><member type="way" ref="485048438" role="outer"/><member type="way" ref="286342034" role="outer"/><member type="way" ref="286342029" role="outer"/><member type="way" ref="286342027" role="outer"/><member type="way" ref="286342031" role="outer"/><member type="way" ref="485048439" role="outer"/><member type="way" ref="485050427" role="outer"/><member type="way" ref="291967375" role="outer"/><member type="way" ref="502262239" role="outer"/><member type="way" ref="286342024" role="outer"/><member type="way" ref="472238985" role="outer"/><member type="way" ref="502262234" role="outer"/><member type="way" ref="472238993" role="outer"/><member type="way" ref="191877594" role="outer"/><member type="way" ref="472236749" role="outer"/><member type="way" ref="191877592" role="outer"/><member type="way" ref="286342012" role="outer"/><member type="way" 

In [15]:
# Let us correct it ! Edit your RE pattern:

qname  = re.sub( r'^.*(<tag k="name" v=")(Quận\s+(\d+|[A-Za-zÀ-ỹ\s]+))("/>.*)$', r'\2', line )
qname 

# Ok? 

'Quận Bình Thạnh'

In [16]:
# Now list all Quan:

quan_lst = []
quan_ref = []

for k in range( len(rel_info) ):
    line = rel_info[k]
    if ( re.match( r'^.*Quận.*$', line, flags=re.IGNORECASE ) is not None ) :
        if ( re.match( r'^.*<tag k="name" v="Quận\s+.*$', line, flags=re.IGNORECASE ) is not None ) :
            
            quan = re.sub( r'^.*(<tag k="name" v=")(Quận\s+(\d+|[A-Za-zÀ-ỹ\s]+))("/>.*)$', r'\2', line )
            if ( quan is not None ):
                quan_lst.append ( quan )
                quan_ref.append ( k )

            #print ( k, rel_ref[k], line, '\n' )
            print ( ' relation reference = %12d --> %s'  % (k, quan) )

 relation reference =           47 --> Quận 1
 relation reference =           57 --> Quận 7
 relation reference =           68 --> Quận 4
 relation reference =          111 --> Quận Bình Thạnh
 relation reference =          134 --> Quận 3
 relation reference =          135 --> Quận 5
 relation reference =          157 --> Quận Phú Nhuận
 relation reference =          260 --> Quận 10
 relation reference =          267 --> Quận 6
 relation reference =          281 --> Quận Tân Phú
 relation reference =          293 --> Quận Tân Bình
 relation reference =          295 --> Quận 11
 relation reference =          312 --> Quận 8
 relation reference =          351 --> Quận Gò Vấp
 relation reference =          368 --> Quận Bình Tân
 relation reference =          379 --> Quận 12


In [17]:
for k in range ( len(quan_ref) ) :
    print ( '%0.3d-th:  relation reference = %8s, %-30s' %( k+1,  quan_ref[k], quan_lst[k] ) )

001-th:  relation reference =       47, Quận 1                        
002-th:  relation reference =       57, Quận 7                        
003-th:  relation reference =       68, Quận 4                        
004-th:  relation reference =      111, Quận Bình Thạnh               
005-th:  relation reference =      134, Quận 3                        
006-th:  relation reference =      135, Quận 5                        
007-th:  relation reference =      157, Quận Phú Nhuận                
008-th:  relation reference =      260, Quận 10                       
009-th:  relation reference =      267, Quận 6                        
010-th:  relation reference =      281, Quận Tân Phú                  
011-th:  relation reference =      293, Quận Tân Bình                 
012-th:  relation reference =      295, Quận 11                       
013-th:  relation reference =      312, Quận 8                        
014-th:  relation reference =      351, Quận Gò Vấp                   
015-th

# We do not have relation data for Quan 2, 9. Why? 
### Because they are not available for now. Verify this fact directly on the OSM website please !

# Next, we extract details of a Quận.
# Note: 
+ ## Quận is a relation
+ ## A relation may include many other relations, and ways, nodes, tags

In [18]:
# Pick up some Quan, e.g.:

#k = 1 # Quan 7
#k = 2 # Quan 4
#k = 5 # Quan 5
k = 10  # Quan Tan Binh

fquan = quan_lst[k]

# reference (index) of Quan:
iquan = quan_ref[k]

# contents of Quan:
rquan = rel_info [ iquan ]

print ( "---> ", fquan, "\n", k, "\n", iquan, "\n", rquan, "\n" )

--->  Quận Tân Bình 
 10 
 293 
 <member type="way" ref="463565950" role="outer"/><member type="way" ref="463565952" role="outer"/><member type="way" ref="1011049448" role="outer"/><member type="way" ref="468274375" role="outer"/><member type="way" ref="463570372" role="outer"/><member type="way" ref="418975307" role="outer"/><member type="way" ref="418922107" role="outer"/><member type="way" ref="495849089" role="outer"/><member type="way" ref="495849090" role="outer"/><member type="way" ref="496065784" role="outer"/><member type="way" ref="418891927" role="outer"/><member type="way" ref="418891936" role="outer"/><member type="way" ref="418891928" role="outer"/><member type="way" ref="418891930" role="outer"/><member type="way" ref="289931217" role="outer"/><member type="way" ref="467125310" role="outer"/><member type="way" ref="468659632" role="outer"/><member type="way" ref="468749913" role="outer"/><member type="way" ref="468665809" role="outer"/><member type="way" ref="468665807" 

In [19]:
# Getting all tags of Quan:

tags = []
for x in re.findall( r'<tag\b[^>]*?/>', rquan ):
    tags.append( x )
#
tags

# Explanation:
#  <tag: Matches the literal opening <tag.
#  \b:  Ensures that tag is a whole word (not part of another word like tagged).
#  [^>]*?: Matches any characters except > (non-greedy) to capture attributes within the tag.
#  />: Matches the self-closing tag ending.

['<tag k="admin_level" v="6"/>',
 '<tag k="boundary" v="administrative"/>',
 '<tag k="is_in:city" v="Ho Chi Minh City"/>',
 '<tag k="name" v="Quận Tân Bình"/>',
 '<tag k="name:en" v="Tan Binh District"/>',
 '<tag k="name:ko" v="떤빈군"/>',
 '<tag k="name:vi" v="Quận Tân Bình"/>',
 '<tag k="name:zh" v="新平郡"/>',
 '<tag k="type" v="boundary"/>',
 '<tag k="wikidata" v="Q2200028"/>']

In [20]:
print ( len(tags) )

10


In [21]:
# Getting all members of Quan:

members = []
for x in re.findall( r'<member\b[^>]*?/>', rquan ):
    members.append( x )
#
members

['<member type="way" ref="463565950" role="outer"/>',
 '<member type="way" ref="463565952" role="outer"/>',
 '<member type="way" ref="1011049448" role="outer"/>',
 '<member type="way" ref="468274375" role="outer"/>',
 '<member type="way" ref="463570372" role="outer"/>',
 '<member type="way" ref="418975307" role="outer"/>',
 '<member type="way" ref="418922107" role="outer"/>',
 '<member type="way" ref="495849089" role="outer"/>',
 '<member type="way" ref="495849090" role="outer"/>',
 '<member type="way" ref="496065784" role="outer"/>',
 '<member type="way" ref="418891927" role="outer"/>',
 '<member type="way" ref="418891936" role="outer"/>',
 '<member type="way" ref="418891928" role="outer"/>',
 '<member type="way" ref="418891930" role="outer"/>',
 '<member type="way" ref="289931217" role="outer"/>',
 '<member type="way" ref="467125310" role="outer"/>',
 '<member type="way" ref="468659632" role="outer"/>',
 '<member type="way" ref="468749913" role="outer"/>',
 '<member type="way" ref="4

In [22]:
print ( len(members)  ) 

47


In [23]:
# HOWEVER, we only need the body of text, excluding the keyword <tag ... >.
# Let's make it better.

# Again, getting all tags of Quan:
        
tags = []
for x in re.findall( r'<tag\b(.*?)\/>', rquan ):
    tags.append( trim(x) )
#
print ( len(tags) )
tags

# Explanation:
#  <tag\b: Matches the literal opening <tag and ensures tag is a whole word.
#  (.*?):  Captures everything between <tag and />. The ? ensures a non-greedy match, 
#          so it stops at the first />.
#  \/>:    Matches the self-closing />.

10


['k="admin_level" v="6"',
 'k="boundary" v="administrative"',
 'k="is_in:city" v="Ho Chi Minh City"',
 'k="name" v="Quận Tân Bình"',
 'k="name:en" v="Tan Binh District"',
 'k="name:ko" v="떤빈군"',
 'k="name:vi" v="Quận Tân Bình"',
 'k="name:zh" v="新平郡"',
 'k="type" v="boundary"',
 'k="wikidata" v="Q2200028"']

In [24]:
def quan_has_tags ( rquan ):
    tags = []
    for x in re.findall( r'<tag\b(.*?)\/>', rquan ):
        tags.append( trim(x) )
    return tags 
#

In [25]:
quan_has_tags ( rquan )

['k="admin_level" v="6"',
 'k="boundary" v="administrative"',
 'k="is_in:city" v="Ho Chi Minh City"',
 'k="name" v="Quận Tân Bình"',
 'k="name:en" v="Tan Binh District"',
 'k="name:ko" v="떤빈군"',
 'k="name:vi" v="Quận Tân Bình"',
 'k="name:zh" v="新平郡"',
 'k="type" v="boundary"',
 'k="wikidata" v="Q2200028"']

In [26]:
# Again, getting all members of Quan:

members = []
for x in re.findall( r'<member\b(.*?)\/>', rquan ):
    members.append( trim(x) )
#
print ( len(members)  ) 
members

47


['type="way" ref="463565950" role="outer"',
 'type="way" ref="463565952" role="outer"',
 'type="way" ref="1011049448" role="outer"',
 'type="way" ref="468274375" role="outer"',
 'type="way" ref="463570372" role="outer"',
 'type="way" ref="418975307" role="outer"',
 'type="way" ref="418922107" role="outer"',
 'type="way" ref="495849089" role="outer"',
 'type="way" ref="495849090" role="outer"',
 'type="way" ref="496065784" role="outer"',
 'type="way" ref="418891927" role="outer"',
 'type="way" ref="418891936" role="outer"',
 'type="way" ref="418891928" role="outer"',
 'type="way" ref="418891930" role="outer"',
 'type="way" ref="289931217" role="outer"',
 'type="way" ref="467125310" role="outer"',
 'type="way" ref="468659632" role="outer"',
 'type="way" ref="468749913" role="outer"',
 'type="way" ref="468665809" role="outer"',
 'type="way" ref="468665807" role="outer"',
 'type="way" ref="468749919" role="outer"',
 'type="way" ref="468652914" role="outer"',
 'type="way" ref="467114640" ro

In [27]:
def quan_has_members ( rquan ):
    members = []
    for x in re.findall( r'<member\b(.*?)\/>', rquan ):
        members.append( trim(x) )
    return members 
#

In [28]:
quan_has_members ( rquan )

['type="way" ref="463565950" role="outer"',
 'type="way" ref="463565952" role="outer"',
 'type="way" ref="1011049448" role="outer"',
 'type="way" ref="468274375" role="outer"',
 'type="way" ref="463570372" role="outer"',
 'type="way" ref="418975307" role="outer"',
 'type="way" ref="418922107" role="outer"',
 'type="way" ref="495849089" role="outer"',
 'type="way" ref="495849090" role="outer"',
 'type="way" ref="496065784" role="outer"',
 'type="way" ref="418891927" role="outer"',
 'type="way" ref="418891936" role="outer"',
 'type="way" ref="418891928" role="outer"',
 'type="way" ref="418891930" role="outer"',
 'type="way" ref="289931217" role="outer"',
 'type="way" ref="467125310" role="outer"',
 'type="way" ref="468659632" role="outer"',
 'type="way" ref="468749913" role="outer"',
 'type="way" ref="468665809" role="outer"',
 'type="way" ref="468665807" role="outer"',
 'type="way" ref="468749919" role="outer"',
 'type="way" ref="468652914" role="outer"',
 'type="way" ref="467114640" ro

# How to extract all the Way references in this Quan?


# How to extract all the Node references in this Quan?


# How to extract all the Relation references in this Quan?

+ ## Once you have a way reference, searching the table of Ways, you will get all its Node references
    + ### Once you have a node reference, searching the table of Nodes, you will get its latitude, longitude. Therefore, you can draw that way. 
    

In [29]:
# We need this function:

def get_ref ( ele ):
    k  = re.sub( r'^.*\s+ref="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', ele )
    #return int(k)
    return np.dtype('int64').type( k )

In [30]:
# First, we do it for Way references:

for k in range ( len(members) ):
    if ( re.split( ' ', members[k] )[0] == 'type="way"' ) :
        j = get_ref ( members[k] )
        print ( k, members[k], ' --> ', j )
#
# Can you see this list of references?

0 type="way" ref="463565950" role="outer"  -->  463565950
1 type="way" ref="463565952" role="outer"  -->  463565952
2 type="way" ref="1011049448" role="outer"  -->  1011049448
3 type="way" ref="468274375" role="outer"  -->  468274375
4 type="way" ref="463570372" role="outer"  -->  463570372
5 type="way" ref="418975307" role="outer"  -->  418975307
6 type="way" ref="418922107" role="outer"  -->  418922107
7 type="way" ref="495849089" role="outer"  -->  495849089
8 type="way" ref="495849090" role="outer"  -->  495849090
9 type="way" ref="496065784" role="outer"  -->  496065784
10 type="way" ref="418891927" role="outer"  -->  418891927
11 type="way" ref="418891936" role="outer"  -->  418891936
12 type="way" ref="418891928" role="outer"  -->  418891928
13 type="way" ref="418891930" role="outer"  -->  418891930
14 type="way" ref="289931217" role="outer"  -->  289931217
15 type="way" ref="467125310" role="outer"  -->  467125310
16 type="way" ref="468659632" role="outer"  -->  468659632
17 ty

In [31]:
def quan_has_member_ref ( rquan, type='node' ):
#    
#   type='node'
#   type='way'
#   type='relation'
#
    def get_ref ( ele ):
        k  = re.sub( r'^.*\s+ref="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', ele )
        return np.dtype('int64').type( k )
#    
    t = str( 'type="' + str(type) + '"' ) 
    m = quan_has_members ( rquan )
    r = []
    for k in range ( len(m) ):
        if ( re.split( ' ', members[k] )[0] == t ) :
            j = get_ref ( members[k] )
            r.append(j)
    return r 
#

In [32]:
def quan_has_ref_of_way ( rquan ):    
    return sorted(quan_has_member_ref ( rquan, type="way" ))
#

In [33]:
quan_has_ref_of_way ( rquan )

[289931217,
 418891927,
 418891928,
 418891930,
 418891936,
 418922107,
 418975307,
 463565950,
 463565951,
 463565952,
 463570372,
 466933217,
 466939789,
 466939792,
 466939796,
 467105267,
 467114640,
 467125310,
 467196322,
 467199746,
 468274375,
 468652914,
 468659632,
 468665807,
 468665809,
 468749913,
 468749919,
 495849089,
 495849090,
 496065784,
 1011049448]

In [34]:
# Second, we do it for Nodes references:

for k in range ( len(members) ):
    if ( re.split( ' ', members[k] )[0] == 'type="node"' ) :
        j = get_ref ( members[k] )
        print ( k, members[k], ' --> ', j )
#
# Can you see this list of references?

46 type="node" ref="11214914766" role="label"  -->  11214914766


In [35]:
def quan_has_ref_of_node ( rquan ):
    return sorted(quan_has_member_ref ( rquan, type='node' ))
#    

In [36]:
quan_has_ref_of_node ( rquan )

[11214914766]

In [37]:
# Last, we do it for Relation references:

for k in range ( len(members) ):
    if ( re.split( ' ', members[k] )[0] == 'type="relation"' ) :
        j = get_ref ( members[k] )
        print ( k, members[k], ' --> ', j )
#
# Can you see this list of references?

31 type="relation" ref="6892400" role="subarea"  -->  6892400
32 type="relation" ref="6892589" role="subarea"  -->  6892589
33 type="relation" ref="6892590" role="subarea"  -->  6892590
34 type="relation" ref="6893381" role="subarea"  -->  6893381
35 type="relation" ref="6893382" role="subarea"  -->  6893382
36 type="relation" ref="6893383" role="subarea"  -->  6893383
37 type="relation" ref="6893384" role="subarea"  -->  6893384
38 type="relation" ref="6893385" role="subarea"  -->  6893385
39 type="relation" ref="6846178" role="subarea"  -->  6846178
40 type="relation" ref="6846133" role="subarea"  -->  6846133
41 type="relation" ref="6893386" role="subarea"  -->  6893386
42 type="relation" ref="6893431" role="subarea"  -->  6893431
43 type="relation" ref="6893432" role="subarea"  -->  6893432
44 type="relation" ref="6893433" role="subarea"  -->  6893433
45 type="relation" ref="6893434" role="subarea"  -->  6893434


In [38]:
def quan_has_ref_of_relation ( rquan ):
    return sorted(quan_has_member_ref ( rquan, type='relation' ))
#    

In [39]:
x = quan_has_ref_of_relation ( rquan )
x

[6846133,
 6846178,
 6892400,
 6892589,
 6892590,
 6893381,
 6893382,
 6893383,
 6893384,
 6893385,
 6893386,
 6893431,
 6893432,
 6893433,
 6893434]

In [40]:
sorted(x)

[6846133,
 6846178,
 6892400,
 6892589,
 6892590,
 6893381,
 6893382,
 6893383,
 6893384,
 6893385,
 6893386,
 6893431,
 6893432,
 6893433,
 6893434]

In [41]:
def quan_has_refs_n_tags ( rquan ):
    return \
    quan_has_ref_of_node (rquan), \
    quan_has_ref_of_way ( rquan ), \
    quan_has_ref_of_relation ( rquan ), \
    quan_has_tags ( rquan )
#

In [42]:
quan_has_refs_n_tags ( rquan ) 

([11214914766],
 [289931217,
  418891927,
  418891928,
  418891930,
  418891936,
  418922107,
  418975307,
  463565950,
  463565951,
  463565952,
  463570372,
  466933217,
  466939789,
  466939792,
  466939796,
  467105267,
  467114640,
  467125310,
  467196322,
  467199746,
  468274375,
  468652914,
  468659632,
  468665807,
  468665809,
  468749913,
  468749919,
  495849089,
  495849090,
  496065784,
  1011049448],
 [6846133,
  6846178,
  6892400,
  6892589,
  6892590,
  6893381,
  6893382,
  6893383,
  6893384,
  6893385,
  6893386,
  6893431,
  6893432,
  6893433,
  6893434],
 ['k="admin_level" v="6"',
  'k="boundary" v="administrative"',
  'k="is_in:city" v="Ho Chi Minh City"',
  'k="name" v="Quận Tân Bình"',
  'k="name:en" v="Tan Binh District"',
  'k="name:ko" v="떤빈군"',
  'k="name:vi" v="Quận Tân Bình"',
  'k="name:zh" v="新平郡"',
  'k="type" v="boundary"',
  'k="wikidata" v="Q2200028"'])

In [43]:
#
# Finally, all what we can extract from a Quan (a relation) are as follows:

qn, qw, qr, qt = quan_has_refs_n_tags ( rquan ) 

print ( '\n*** Node ref:\n\t', qn ) 
print ( '\n*** Way ref:\n\t', qw ) 
print ( '\n*** Relation ref:\n\t', qr ) 
print ( '\n*** Tags:\n\t', qt ) 


*** Node ref:
	 [11214914766]

*** Way ref:
	 [289931217, 418891927, 418891928, 418891930, 418891936, 418922107, 418975307, 463565950, 463565951, 463565952, 463570372, 466933217, 466939789, 466939792, 466939796, 467105267, 467114640, 467125310, 467196322, 467199746, 468274375, 468652914, 468659632, 468665807, 468665809, 468749913, 468749919, 495849089, 495849090, 496065784, 1011049448]

*** Relation ref:
	 [6846133, 6846178, 6892400, 6892589, 6892590, 6893381, 6893382, 6893383, 6893384, 6893385, 6893386, 6893431, 6893432, 6893433, 6893434]

*** Tags:
	 ['k="admin_level" v="6"', 'k="boundary" v="administrative"', 'k="is_in:city" v="Ho Chi Minh City"', 'k="name" v="Quận Tân Bình"', 'k="name:en" v="Tan Binh District"', 'k="name:ko" v="떤빈군"', 'k="name:vi" v="Quận Tân Bình"', 'k="name:zh" v="新平郡"', 'k="type" v="boundary"', 'k="wikidata" v="Q2200028"']


# Part 2. NODES

In [44]:
from datascience import *
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# If you don't love the datascience library. I don't mind. 
# Do your own but robust method! 

In [45]:
t_node = Table().with_columns(
    'ref', node_ref,
    'lat', node_lat,
    'lon', node_lon
).sort( 'ref' )

print ( t_node.num_rows  )

t_node.show(3)

2558903


ref,lat,lon
91665273,10.9703,106.892
91665277,10.951,106.872
91665279,10.9434,106.868


In [46]:
#t_node.to_csv ( 'TpHCM_nodes_ref_lat_lon.csv' )

# How to fast search a row of table based on index (i.e. reference) ? 
## Assumming that 
+ ## the sequence of indices is ascending,
+ ## Indices are uniquely determined.

In [47]:
# Slow search method

xid = 366238048
t_node.where ( 'ref', are.equal_to( xid ) )

ref,lat,lon
366238048,10.6508,106.736


In [48]:
# Fast binary search method
#
# Ref.: 
#   https://numpy.org/doc/stable/reference/generated/numpy.searchsorted.html
# Usage:
#   numpy.searchsorted ( a, v, side='left', sorter=None )
#     + a: 1-D array_like. Input array. 
#          If sorter is None, then it must be sorted in ascending order,
#          otherwise sorter must be an array of indices that sort it.
#     + v: array_like. Values to insert into a. Here v may have many values.
#     + side   returned index i satisfies
#       left   a[i-1] < v <= a[i]
#       right  a[i-1] <= v < a[i]
#     + sorter: 1-D array_like, optional
#           Optional array of integer indices that sort array a into ascending 
#           order. They are typically the result of argsort.
#           For example: 
#              ia = np.argsort ( a )
#              i = np.searchsorted ( a, v, side='left', sorter=ia )
#

k = np.searchsorted ( node_ref, xid )
print ( k ) 
print ( node_ref[k], node_lat[k], node_lon[k] ) 

696
366238048 10.6507981 106.7362182


In [49]:
#np.argsort?

In [50]:
# Fast binary search method, again:

k = np.searchsorted ( t_node.column('ref'), xid )
print ( k ) 

t_node.take(k).show()

696


ref,lat,lon
366238048,10.6508,106.736


In [51]:
# Another test? 

xid = 12339046869

# Slow method:
t_node.where ( 'ref', are.equal_to( xid ) ).show() 

ref,lat,lon
12339046869,11.0759,106.764


In [52]:
# Fast binary search method:
k = np.searchsorted ( node_ref, xid )
print ( k ) 
print ( node_ref[k], node_lat[k], node_lon[k] ) 

2558896
12339046869 11.0759466 106.7642506


In [53]:
# Fast binary search method, again:

k = np.searchsorted ( t_node.column('ref'), xid )
print ( k ) 

t_node.take(k).show()

2558896


ref,lat,lon
12339046869,11.0759,106.764


In [54]:
# Let us check the nodes included in the relation (above):
# qn, qw, qr, qt = quan_has_refs_n_tags ( rquan ) 

for xid in qn:
    k = np.searchsorted ( node_ref1, xid ) 
    print ( k, node_ref1[k], node_info[k] ) 

61475 11214914766 <tag k="name" v="Tân Bình"/><tag k="place" v="suburb"/>


In [55]:
# Let us check the ways included in the relation (above):
# qn, qw, qr, qt = quan_has_refs_n_tags ( rquan ) 

for xid in qw:
    k = np.searchsorted ( way_ref, xid ) 
    print ( k, '\n ---> ', way_ref[k], way_info[k] ) 

24197 
 --->  289931217 <nd ref="4191172081"/><nd ref="2322648184"/><nd ref="4191172094"/><nd ref="4191172103"/><nd ref="4191172113"/><nd ref="5806611596"/><nd ref="2934668192"/><nd ref="4191172118"/><nd ref="4191172119"/><nd ref="8414354136"/><nd ref="2934668193"/><nd ref="4191172120"/><nd ref="4191172121"/><nd ref="4191172122"/><nd ref="4191172123"/><nd ref="4191172124"/><nd ref="2934668194"/><nd ref="2934668195"/><nd ref="2934668196"/><nd ref="2934668197"/><nd ref="2934668198"/><nd ref="2934668199"/><nd ref="7016368434"/><nd ref="2934668200"/><nd ref="11405918336"/><nd ref="2934670002"/><nd ref="11434275747"/><nd ref="10882554975"/><nd ref="2934670004"/><nd ref="2934670005"/><nd ref="2934670006"/><nd ref="2901394135"/><nd ref="2901394134"/><nd ref="4440166373"/><tag k="admin_level" v="6"/><tag k="boundary" v="administrative"/>
39381 
 --->  418891927 <nd ref="8262886830"/><nd ref="4191172007"/><tag k="admin_level" v="6"/><tag k="boundary" v="administrative"/>
39382 
 --->  418891928

In [56]:
# Let us check the relations included in the relation (above):
# qn, qw, qr, qt = quan_has_refs_n_tags ( rquan ) 

for xid in qr:
    k = np.searchsorted ( rel_ref, xid ) 
    print ( k, '\n ---> ', rel_ref[k], rel_info[k] ) 

283 
 --->  6846133 <member type="node" ref="10167952542" role="label"/><member type="way" ref="463565951" role="outer"/><member type="way" ref="466939789" role="outer"/><member type="way" ref="467196322" role="outer"/><member type="way" ref="467196324" role="outer"/><member type="way" ref="467196331" role="outer"/><member type="way" ref="467196327" role="outer"/><member type="way" ref="467196321" role="outer"/><tag k="admin_level" v="8"/><tag k="boundary" v="administrative"/><tag k="is_in:city" v="Ho Chi Minh City"/><tag k="name" v="Phường 10"/><tag k="name:en" v="Ward 10"/><tag k="name:vi" v="Phường 10"/><tag k="type" v="boundary"/><tag k="wikidata" v="Q2588278"/><tag k="wikipedia" v="vi:Phường 10, Tân Bình"/>
294 
 --->  6846178 <member type="node" ref="10167952543" role="label"/><member type="way" ref="463565950" role="outer"/><member type="way" ref="463565952" role="outer"/><member type="way" ref="1011049448" role="outer"/><member type="way" ref="468274375" role="outer"/><member t

# Conclusion: we should apply the binary search to gain better performance.
## Implementation:

In [57]:

# For table:
def table_search ( t, xid, col=0 ):
#
#   Return a sub-table, namely tout, with only two rows such that
#     tout.column(col).item(1) < xid <= tout.column(col).item(0)
#
    from numpy import argsort, searchsorted
#    
    if ( col == 0 ):
        k = searchsorted( t.column(col), xid )
        return t.take( [ k, k-1 ] )        
    else:
        ia = argsort( t.column(col) )   
        k = searchsorted( t.column(col), xid, sorter=ia )
        return t.take( [  ia[k], ia[k-1] ] )
#


# For array: (general purpose)
def array_search ( a, x, getsorted=True ):
#
#   Return a couple of integers, namely k1 and k2, such that
#     a[k2] < x <= a[k1]
#
    from numpy import argsort, searchsorted
#    
    if ( getsorted ):
        k = searchsorted( a, x )
        k1 = k
        k2 = k-1 
    else:
        ia = argsort( a )   
        k = searchsorted( a, x, sorter=ia )
        k1 = ia[k]
        k2 = ia[k-1] 
#        
    return k1, k2 
#

#  
def find_id_from_ref ( aref, iref, getsorted=True ):
#
#   Return such that aref[k] = iref
#   Usage:
#      k = findref ( aref, iref )
#   Assumption:
#   + iref exists uniquely in aref. 
#
    from numpy import argsort, searchsorted
#    
    if ( getsorted ):
        k = searchsorted( aref, iref )
    else:
        ia = argsort( aref )   
        k = searchsorted( aref, iref, sorter=ia )
#        
    return k
#

In [58]:
x = 12339046869

print( "using table_search: x=", x )
table_search ( t_node, x ).show()

print( "using array_search:" )
t_node.take( array_search ( t_node.column(0), x )  ).show()

x = 366238048

print( "using table_search: x =", x )
table_search ( t_node, x ).show()

print( "using array_search:" )
t_node.take( array_search ( t_node.column(0), x )  ).show()

using table_search: x= 12339046869


ref,lat,lon
12339046869,11.0759,106.764
12339011245,10.8253,106.773


using array_search:


ref,lat,lon
12339046869,11.0759,106.764
12339011245,10.8253,106.773


using table_search: x = 366238048


ref,lat,lon
366238048,10.6508,106.736
366238047,10.6498,106.737


using array_search:


ref,lat,lon
366238048,10.6508,106.736
366238047,10.6498,106.737


In [59]:
x = 366238000

print( "using table_search: x =", x, ' (where x is not in the table)' )
table_search ( t_node, x, col='ref' ).show()

print( "using array_search:" )
t_node.take( array_search ( t_node.column('ref'), x ) ).show()

using table_search: x = 366238000  (where x is not in the table)


ref,lat,lon
366238046,10.6484,106.738
366237947,10.6435,106.743


using array_search:


ref,lat,lon
366238046,10.6484,106.738
366237947,10.6435,106.743


In [60]:
x = 107

print( "using table_search: x =", x )
table_search ( t_node, x, col='lon' ).show()

print( "using array_search:" )
t_node.take( array_search ( t_node.column('lon'), x, getsorted=False ) ).show()

using table_search: x = 107


ref,lat,lon
9391394565,10.9752,107
12174908438,10.8046,107


using array_search:


ref,lat,lon
9391394565,10.9752,107
12174908438,10.8046,107


In [61]:
t_node_info = Table().with_columns(
    'ref', node_ref1,
    'info', node_info
)

print ( t_node_info.num_rows )

t_node_info.show(3)

75212


ref,info
305327873,
305328007,"<tag k=""crossing:bel ..."
305328010,


In [62]:
#t_node_info.to_csv ( 'TpHCM_nodes_ref_info.csv' )

# For a node in our interest, how to get its latitude and longitude quickly?

In [63]:
Np = 20
np.random.choice( np.arange(0, len(node_ref)), size=Np )

array([1307008,  563972,  498108, 1958184, 2506922, 2228131,   58214,
        966890, 1594613,  513031,  834369,  551203, 1799545, 2038911,
       1514086, 1789316,  103116,  150448,  451885,  885470])

In [64]:
# Demo:

Np = 5

for k in np.random.choice( np.arange(0, len(node_ref1)), size=Np ):
    
    print ( "\n*** Information and (lat,lon)-coordinates of the %0.3d-th node:\n"%( k+1 ) ) 
    
    Tx = t_node_info.take ( k )
    ix = Tx.column(0).item(0)

    print ( ix, Tx.column(1).item(0), '\n' )
    
    print( "using table_search: iref =", ix )
    Ty = table_search ( t_node, ix )
    Ty.show()

    print( "using array_search:" )
    Tz = t_node.take( array_search ( t_node.column('ref'), ix, getsorted=False ) )
    Tz.show()


*** Information and (lat,lon)-coordinates of the 22844-th node:

5272884338 <tag k="addr:housenumber" v="1c"/><tag k="note" v="HD_PHUOCBINH_9/12/2017"/> 

using table_search: iref = 5272884338


ref,lat,lon
5272884338,10.8169,106.771
5272884337,10.815,106.771


using array_search:


ref,lat,lon
5272884338,10.8169,106.771
5272884337,10.815,106.771



*** Information and (lat,lon)-coordinates of the 47900-th node:

9265379579 <tag k="highway" v="traffic_signals"/><tag k="traffic_signals" v="signal"/><tag k="traffic_signals:countdown" v="yes"/> 

using table_search: iref = 9265379579


...

ref,lat,lon
8269126426,10.7909,106.724
8269126425,10.7909,106.724



*** Information and (lat,lon)-coordinates of the 42704-th node:

6800280720 <tag k="addr:housenumber" v="96"/><tag k="addr:street" v="Đường Đặng Thùy Trâm"/> 

using table_search: iref = 6800280720


ref,lat,lon
6800280720,10.8282,106.703
6800280719,10.8343,106.704


using array_search:


ref,lat,lon
6800280720,10.8282,106.703
6800280719,10.8343,106.704


In [65]:
xlat = 10.5
Tz = t_node.take( array_search ( t_node.column('lat'), xlat, getsorted=False ) )
Tz.show()

ref,lat,lon
6465607190,10.5,106.322
7248433366,10.5,106.413


# How to extract nodes in a bounding box based on latitude and longitude?

In [66]:
def sort_data2 ( xi, yi ):
#
#   Sort sequence of the couples ( xi[j], yi[j] ) such that x is increased.
#   Here y may not be numbers, e.g., it may be strings.
#   Syntax: xs, ys = sort_data2 ( xi, yi )
#
#   Warning: this routine may crash your computer as the input xi, yi
#   are so large. 
#
    from numpy import argsort, size 
#
    js = argsort ( xi ) 
    n = size(xi)    
    xtmp = [] 
    ytmp = []
    for j in range (0,n):
        xtmp.append( xi[js[j]] )
        ytmp.append( yi[js[j]] )
    return xtmp, ytmp 
#

def sort_data3 ( xi, yi, zi ):
#
#   Sort sequence of the triple ( xi[j], yi[j], zi[j] ) such that x is increased.
#   Here y may not be numbers, e.g., it may be strings.
#   Syntax: xs, ys, zs = sort_data3 ( xi, yi, zi )
#
#   Warning: this routine may crash your computer as the input xi, yi, zi
#   are so large. 
#
    from numpy import argsort, size 
#    
    js = argsort ( xi ) 
    n = size(xi)    
    xtmp = [] 
    ytmp = []
    ztmp = []
    for j in range (0,n):
        xtmp.append( xi[js[j]] )
        ytmp.append( yi[js[j]] )
        ztmp.append( zi[js[j]] )
    return xtmp, ytmp, ztmp  
#

In [139]:

def get_ref_from_values ( aref, aval, valmin, valmax ):
#
#   Return a sequence of references, namely iref, for the triple (aref[k], aval[k]) 
#   such that 
#     valmin <= aval[k] <= valmax
#    
    from numpy import argsort, searchsorted, append
#    
    js = argsort ( aval ) 
#
#   Return i such that 
#   side='right': a[i-1] <= v < a[i]
#   side='left':  a[i-1] < v <= a[i]
#
#   a[kmax] <= valmax: 
    kmax = searchsorted( aval, valmax, side='right', sorter=js ) - 1     
#   valmin <= a[kmin]:
    kmin = searchsorted( aval, valmin, side='left', sorter=js ) 
#        
    list_js = [] 
    for j in range (kmin, kmax+1):
        list_js.append ( js[j] )
#        
    return np.dtype('int64').type(aref) [ list_js ]
#


def get_values_from_ref ( aval, aref, iref ):
#    
    from numpy import array, size     
#
    if ( size(iref) == 1 ):        
        iid = find_id_from_ref( np.dtype('int64').type(aref), 
                            iref, getsorted=True)        
#        
#       WARNING: this command may crash your program as aval is so huge, e.g., aval=way_info.        
        return array(aval) [ iid ]
#
    else:
#
#       b = []
#       for j in range( len(iref) ):
#           b = append( b, aval [ find_id_from_ref ( aref, iref[j], getsorted=True ) ] ) 
#       return b 
#
        list_id = [] 
        for j in range( size(iref) ):
            list_id.append( find_id_from_ref( np.dtype('int64').type(aref), 
                            iref[j], getsorted=True) ) 
#            
#       WARNING: this command may crash your program as aval is so huge, e.g., aval=way_info.
        return array(aval) [ list_id ]
#

def get_inform_from_ref ( ainfo, aref, iref ):
#    
    from numpy import array, size     
#
    if ( size(iref) == 1 ):        
        iid = find_id_from_ref( np.dtype('int64').type(aref), 
                            iref, getsorted=True)        
#        
#       WARNING: this command may crash your program as ainfo is so huge, e.g., ainfo=way_info.        
#       return array(ainfo) [ iid ]
#
        return ainfo[iid] 
#
    else:
#
#       b = []
#       for j in range( len(iref) ):
#           b = append( b, aval [ find_id_from_ref ( aref, iref[j], getsorted=True ) ] ) 
#       return b 
#
        list_id = [] 
        for j in range( size(iref) ):
            list_id.append( find_id_from_ref( np.dtype('int64').type(aref), 
                            iref[j], getsorted=True) ) 
#            
#       WARNING: this command may crash your program as aval is so huge, e.g., aval=way_info.
#       return array(ainfo) [ list_id ]
#
        aout = [] 
        for k in list_id:
            aout.append ( ainfo[k] )
#
        return aout 
#

In [68]:
arr = np.array([10, 20, 30, 40, 50])
index_list = [1, 3, 0]  # Indices to extract
extracted_elements = arr[index_list]  
print(extracted_elements)

[20 40 10]


In [69]:
s = []
s = np.append( s, 'a1' )
s = np.append( s, 'a23' )
s

array(['a1', 'a23'],
      dtype='<U32')

In [70]:
z = 3
z = np.dtype('int64').type(z)
z

3

In [71]:
np.size(z)

1

In [72]:
#np.searchsorted?

In [73]:
West, East, South, North = np.min( node_lon ), np.max( node_lon ), np.min( node_lat ), np.max( node_lat )

print ( '%20s %20s %20s' % ( '', North, ''  ) ) 
print ( '%20s %20s %20s' % ( West, '', East ) ) 
print ( '%20s %20s %20s' % ( '', South, ''  ) ) 

                               11.0909999                     
         106.3090014                               107.1519998
                               10.4770002                     


In [74]:
# Let us try the first step: 

latmin = 10.5
latmax = 10.7

iref = get_ref_from_values ( node_ref, node_lat, latmin, latmax )

print ( 'len(iref) =', len(iref) )


n = 50
k = 10
jref = iref[k:k+n]

print ( 'len(jref) =', len(jref) )
print ( 'jref =', jref )


print ( '\nref:', get_values_from_ref( node_ref, node_ref, jref ), '<-- it is jref itself !' )
print ( '\nlat:', get_values_from_ref( node_lat, node_ref, jref )  )
print ( '\nlon:', get_values_from_ref( node_lon, node_ref, jref )  )

len(iref) = 436168
len(jref) = 50
jref = [10576753661 12250999662  8814099428 11806463235  7282626427  3282002618
  8657842053  7248122596  7807352030  6411577769  8657842052  8382562242
  2643958153 11627493123  8819986084  8657842064  4890916652  8819986085
 10921303243  8814251802  4791472401  3720970916   367606684 10289120927
  7813618295 12250999636 10750579033  2724873876   367543055  2652146588
 10706283945  8814088033 10136397408  8382560187  2852297458  1588180604
  6291003806  8814377828   367343161  2847458420   367543053  6465607400
  7810938174  2643958146  8819986086   367543064   367440411  8814286279
   367542958   367606692]

ref: [10576753661 12250999662  8814099428 11806463235  7282626427  3282002618
  8657842053  7248122596  7807352030  6411577769  8657842052  8382562242
  2643958153 11627493123  8819986084  8657842064  4890916652  8819986085
 10921303243  8814251802  4791472401  3720970916   367606684 10289120927
  7813618295 12250999636 10750579033  2724873876   

In [75]:
# Now we combine two steps:

latmin = 10.5
latmax = 10.7

lonmin = 106.6
lonmax = 106.8

latref = get_ref_from_values ( node_ref, node_lat, latmin, latmax )

print ( 'len(latref) =', len(latref) )

lonref = get_ref_from_values ( node_ref, node_lon, lonmin, lonmax )

print ( 'len(lonref) =', len(lonref) )

len(latref) = 436168
len(lonref) = 1258100


In [76]:
# How to collect the same references in distinct arrays ?

# Create two arrays

n1 = np.array([5, 10, 15, 20])
n2 = np.array([5, 15, 30, 40])
 
print("Iterating array1 ...")
print( n1 )

print ("\nIterating array2 " )
print( n2 )


# Find intersection using the intersect1d() method

resarr = np.intersect1d ( n1, n2 )
print( "\nIntersection = \n", resarr )

Iterating array1 ...
[ 5 10 15 20]

Iterating array2 
[ 5 15 30 40]

Intersection = 
 [ 5 15]


In [77]:
# Now we collect references of nodes, namely kref, which remains on the domain 
# specified by [latmin, latmax] x [lonmin, lonmax].

kref = np.intersect1d ( latref, lonref )

print ( 'len(kref) =', len(kref) )
print ( 'kref =', kref )

len(kref) = 126095
kref = [  366221537   366221543   366221545 ..., 12273069708 12273069709
 12273069710]


In [78]:
print ( '%20s %20s %20s' % ( '', North, ''  ) ) 
print ( '%20s %20s %20s' % ( West, '', East ) ) 
print ( '%20s %20s %20s' % ( '', South, ''  ) ) 

k1 = len(kref) - 10
k2 = k1 + 10 

print ( '\nlat:', get_values_from_ref( node_lat, node_ref, kref[k1:k2] )  )
print ( '\nlon:', get_values_from_ref( node_lon, node_ref, kref[k1:k2] )  )

                               11.0909999                     
         106.3090014                               107.1519998
                               10.4770002                     

lat: [ 10.6318658  10.6416365  10.6976368  10.6609972  10.6449538  10.6567198
  10.6919236  10.6836921  10.6836042  10.6838974]

lon: [ 106.7958938  106.7894342  106.7115816  106.6199476  106.6082211
  106.6152686  106.7424009  106.6506882  106.6509554  106.6507551]


# It takes a quite long time to run because the lists of longitudes and latitudes are huge. 
# We should focus only on a specific region and limit the lists on that domain. 

In [79]:
alat = get_values_from_ref( node_lat, node_ref, kref ) 
alat

array([ 10.5287735,  10.5267306,  10.5266134, ...,  10.6836921,
        10.6836042,  10.6838974])

In [80]:
alon = get_values_from_ref( node_lon, node_ref, kref ) 
alon

array([ 106.7654707,  106.7581918,  106.7577539, ...,  106.6506882,
        106.6509554,  106.6507551])

In [81]:
w_node = Table().with_columns(
    'ref', kref,
    'lat', alat,  
    'lon', alon
)

print ( w_node.num_rows )

w_node.show(3)

126095


ref,lat,lon
366221537,10.5288,106.765
366221543,10.5267,106.758
366221545,10.5266,106.758


In [82]:
aref1 = get_values_from_ref( node_ref1, node_ref1, kref )
aref1

array([  366367389,   366367389,   366367389, ..., 12276868136,
       12276868136, 12276868136])

In [83]:
ainfo = get_inform_from_ref(  node_info, node_ref1, kref ) 
ainfo

['<tag k="barrier" v="toll_booth"/><tag k="charge" v="27000 VND;41000 VND;58000 VND;108000VND;152000 VND"/><tag k="fee" v="yes"/><tag k="lanes" v="4"/><tag k="motorcycle" v="no"/>',
 '<tag k="barrier" v="toll_booth"/><tag k="charge" v="27000 VND;41000 VND;58000 VND;108000VND;152000 VND"/><tag k="fee" v="yes"/><tag k="lanes" v="4"/><tag k="motorcycle" v="no"/>',
 '<tag k="barrier" v="toll_booth"/><tag k="charge" v="27000 VND;41000 VND;58000 VND;108000VND;152000 VND"/><tag k="fee" v="yes"/><tag k="lanes" v="4"/><tag k="motorcycle" v="no"/>',
 '<tag k="barrier" v="toll_booth"/><tag k="charge" v="27000 VND;41000 VND;58000 VND;108000VND;152000 VND"/><tag k="fee" v="yes"/><tag k="lanes" v="4"/><tag k="motorcycle" v="no"/>',
 '<tag k="barrier" v="toll_booth"/><tag k="charge" v="27000 VND;41000 VND;58000 VND;108000VND;152000 VND"/><tag k="fee" v="yes"/><tag k="lanes" v="4"/><tag k="motorcycle" v="no"/>',
 '<tag k="barrier" v="toll_booth"/><tag k="charge" v="27000 VND;41000 VND;58000 VND;108000

In [84]:
w_node_info = Table().with_columns(
    'ref', aref1,
    'info', ainfo
)

print ( w_node_info.num_rows )

w_node_info.show(3)

126095


ref,info
366367389,"<tag k=""charge"" v=""2700 ..."
366367389,"<tag k=""charge"" v=""2700 ..."
366367389,"<tag k=""charge"" v=""2700 ..."


# Can we wait? Is it too slow? How to solve this issue? 
## Don't worry if we can't solve this issue right away. We still have a feasible solution. Since we won't be searching through many elements at a time, searching for a few elements in large arrays remains practical.

In [85]:
# So far, remind that what we have extracted from a Quan (a relation) using this command 
#     qn, qw, qr, qt = quan_has_refs_n_tags ( rquan ) 
# are as follows:

print ( '\n*** Node ref:\n\t', qn ) 
print ( '\n*** Way ref:\n\t', qw ) 
print ( '\n*** Relation ref:\n\t', qr ) 
print ( '\n*** Tags:\n\t', qt ) 


*** Node ref:
	 [11214914766]

*** Way ref:
	 [289931217, 418891927, 418891928, 418891930, 418891936, 418922107, 418975307, 463565950, 463565951, 463565952, 463570372, 466933217, 466939789, 466939792, 466939796, 467105267, 467114640, 467125310, 467196322, 467199746, 468274375, 468652914, 468659632, 468665807, 468665809, 468749913, 468749919, 495849089, 495849090, 496065784, 1011049448]

*** Relation ref:
	 [6846133, 6846178, 6892400, 6892589, 6892590, 6893381, 6893382, 6893383, 6893384, 6893385, 6893386, 6893431, 6893432, 6893433, 6893434]

*** Tags:
	 ['k="admin_level" v="6"', 'k="boundary" v="administrative"', 'k="is_in:city" v="Ho Chi Minh City"', 'k="name" v="Quận Tân Bình"', 'k="name:en" v="Tan Binh District"', 'k="name:ko" v="떤빈군"', 'k="name:vi" v="Quận Tân Bình"', 'k="name:zh" v="新平郡"', 'k="type" v="boundary"', 'k="wikidata" v="Q2200028"']


In [86]:
# Before moving on the next part, let us check the location of this Quan

qref = qn[0]

qlat = get_values_from_ref( node_lat, node_ref, qref ) 
qlon = get_values_from_ref( node_lon, node_ref, qref )  

map_center = [ qlat, qlon ]

print ( 'map center point = ( %15.10f,  %15.10f ), node reference = %15d ' % ( qlat, qlon, qref ) ) 


map_osm = folium.Map ( location=map_center, zoom_start=13 )

folium.Marker(
    location=map_center,
    tooltip="Click me!",
    popup="Center point",
    icon=folium.Icon(icon="cloud", color="purple"),
).add_to(map_osm)

map_osm

map center point = (   10.8025830000,   106.6521157000 ), node reference =     11214914766 


# Part 3. WAY

In [87]:
wref = qw [ 0 ]
wref 

289931217

In [88]:
# WARNING: this command will crash your program since aval=way_info is huge. 
#wval =  get_values_from_ref( way_info, way_ref, wref ) 
#wval 

# Let us solve this issue as follows:

wid = find_id_from_ref( np.dtype('int64').type(way_ref), wref, getsorted=True )   
wid 

24197

In [89]:
wval = way_info [ wid ] 
wval 

'<nd ref="4191172081"/><nd ref="2322648184"/><nd ref="4191172094"/><nd ref="4191172103"/><nd ref="4191172113"/><nd ref="5806611596"/><nd ref="2934668192"/><nd ref="4191172118"/><nd ref="4191172119"/><nd ref="8414354136"/><nd ref="2934668193"/><nd ref="4191172120"/><nd ref="4191172121"/><nd ref="4191172122"/><nd ref="4191172123"/><nd ref="4191172124"/><nd ref="2934668194"/><nd ref="2934668195"/><nd ref="2934668196"/><nd ref="2934668197"/><nd ref="2934668198"/><nd ref="2934668199"/><nd ref="7016368434"/><nd ref="2934668200"/><nd ref="11405918336"/><nd ref="2934670002"/><nd ref="11434275747"/><nd ref="10882554975"/><nd ref="2934670004"/><nd ref="2934670005"/><nd ref="2934670006"/><nd ref="2901394135"/><nd ref="2901394134"/><nd ref="4440166373"/><tag k="admin_level" v="6"/><tag k="boundary" v="administrative"/>'

In [90]:
#
# Based the global arrays way_ref and way_info, we define functions to extract data 
# for convenience in use:

def wayid_from_ref ( wref ):
    return find_id_from_ref( np.dtype('int64').type(way_ref), wref, getsorted=True )
#

def wayinfo_from_ref ( wref ):
    return way_info [  wayid_from_ref ( wref ) ]
#

In [91]:
for r in qw: 
    print ( r, wayinfo_from_ref ( r ), '\n' )

289931217 <nd ref="4191172081"/><nd ref="2322648184"/><nd ref="4191172094"/><nd ref="4191172103"/><nd ref="4191172113"/><nd ref="5806611596"/><nd ref="2934668192"/><nd ref="4191172118"/><nd ref="4191172119"/><nd ref="8414354136"/><nd ref="2934668193"/><nd ref="4191172120"/><nd ref="4191172121"/><nd ref="4191172122"/><nd ref="4191172123"/><nd ref="4191172124"/><nd ref="2934668194"/><nd ref="2934668195"/><nd ref="2934668196"/><nd ref="2934668197"/><nd ref="2934668198"/><nd ref="2934668199"/><nd ref="7016368434"/><nd ref="2934668200"/><nd ref="11405918336"/><nd ref="2934670002"/><nd ref="11434275747"/><nd ref="10882554975"/><nd ref="2934670004"/><nd ref="2934670005"/><nd ref="2934670006"/><nd ref="2901394135"/><nd ref="2901394134"/><nd ref="4440166373"/><tag k="admin_level" v="6"/><tag k="boundary" v="administrative"/> 

418891927 <nd ref="8262886830"/><nd ref="4191172007"/><tag k="admin_level" v="6"/><tag k="boundary" v="administrative"/> 

418891928 <nd ref="4191172070"/><nd ref="419117

In [92]:

wref = qw [ 0 ]

wval = wayinfo_from_ref ( wref )
wval 

# Take a look at this:

'<nd ref="4191172081"/><nd ref="2322648184"/><nd ref="4191172094"/><nd ref="4191172103"/><nd ref="4191172113"/><nd ref="5806611596"/><nd ref="2934668192"/><nd ref="4191172118"/><nd ref="4191172119"/><nd ref="8414354136"/><nd ref="2934668193"/><nd ref="4191172120"/><nd ref="4191172121"/><nd ref="4191172122"/><nd ref="4191172123"/><nd ref="4191172124"/><nd ref="2934668194"/><nd ref="2934668195"/><nd ref="2934668196"/><nd ref="2934668197"/><nd ref="2934668198"/><nd ref="2934668199"/><nd ref="7016368434"/><nd ref="2934668200"/><nd ref="11405918336"/><nd ref="2934670002"/><nd ref="11434275747"/><nd ref="10882554975"/><nd ref="2934670004"/><nd ref="2934670005"/><nd ref="2934670006"/><nd ref="2901394135"/><nd ref="2901394134"/><nd ref="4440166373"/><tag k="admin_level" v="6"/><tag k="boundary" v="administrative"/>'

In [93]:
def way_has_tags ( wval ):
    tags = []
    for x in re.findall( r'<tag\b(.*?)\/>', wval ):
        tags.append( trim(x) )
    return tags 
#

def way_has_nodes ( wval ):
    nodes = []
    for x in re.findall( r'<nd\b(.*?)\/>', wval ):
        nodes.append( trim(x) )
    return nodes
#

def way_has_ref_of_node ( wval ):
#
    def get_ref ( ele ):
        k  = re.sub( r'^ref="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"$', r'\1', ele )
        return np.dtype('int64').type( k )
#
    rnodes = []
    for x in way_has_nodes ( wval ):
        rnodes.append( get_ref ( trim(x) ) )
    return np.dtype('int64').type( rnodes )
#

In [94]:
way_has_tags ( wval )

['k="admin_level" v="6"', 'k="boundary" v="administrative"']

In [95]:
wn = way_has_nodes ( wval )
wn 

['ref="4191172081"',
 'ref="2322648184"',
 'ref="4191172094"',
 'ref="4191172103"',
 'ref="4191172113"',
 'ref="5806611596"',
 'ref="2934668192"',
 'ref="4191172118"',
 'ref="4191172119"',
 'ref="8414354136"',
 'ref="2934668193"',
 'ref="4191172120"',
 'ref="4191172121"',
 'ref="4191172122"',
 'ref="4191172123"',
 'ref="4191172124"',
 'ref="2934668194"',
 'ref="2934668195"',
 'ref="2934668196"',
 'ref="2934668197"',
 'ref="2934668198"',
 'ref="2934668199"',
 'ref="7016368434"',
 'ref="2934668200"',
 'ref="11405918336"',
 'ref="2934670002"',
 'ref="11434275747"',
 'ref="10882554975"',
 'ref="2934670004"',
 'ref="2934670005"',
 'ref="2934670006"',
 'ref="2901394135"',
 'ref="2901394134"',
 'ref="4440166373"']

In [96]:
wn[0]

'ref="4191172081"'

In [97]:
ele =  wn[0] 
print ( ele )

re.sub( r'ref="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"$', r'\1',  ele )

ref="4191172081"


'4191172081'

In [98]:
way_has_ref_of_node ( wval )

array([ 4191172081,  2322648184,  4191172094,  4191172103,  4191172113,
        5806611596,  2934668192,  4191172118,  4191172119,  8414354136,
        2934668193,  4191172120,  4191172121,  4191172122,  4191172123,
        4191172124,  2934668194,  2934668195,  2934668196,  2934668197,
        2934668198,  2934668199,  7016368434,  2934668200, 11405918336,
        2934670002, 11434275747, 10882554975,  2934670004,  2934670005,
        2934670006,  2901394135,  2901394134,  4440166373])

In [99]:
def way_has_refs_n_tags ( wval ):
    return \
    way_has_ref_of_node ( wval ), \
    way_has_tags ( wval )
#

In [100]:
way_has_refs_n_tags ( wval )

(array([ 4191172081,  2322648184,  4191172094,  4191172103,  4191172113,
         5806611596,  2934668192,  4191172118,  4191172119,  8414354136,
         2934668193,  4191172120,  4191172121,  4191172122,  4191172123,
         4191172124,  2934668194,  2934668195,  2934668196,  2934668197,
         2934668198,  2934668199,  7016368434,  2934668200, 11405918336,
         2934670002, 11434275747, 10882554975,  2934670004,  2934670005,
         2934670006,  2901394135,  2901394134,  4440166373]),
 ['k="admin_level" v="6"', 'k="boundary" v="administrative"'])

In [101]:
#
# Finally, all what we can extract from a way are as follows:

wn, wt = way_has_refs_n_tags ( wval ) 

print ( '\n*** Node ref:\n\t', wn ) 
print ( '\n*** Tags:\n\t', wt ) 


*** Node ref:
	 [ 4191172081  2322648184  4191172094  4191172103  4191172113  5806611596
  2934668192  4191172118  4191172119  8414354136  2934668193  4191172120
  4191172121  4191172122  4191172123  4191172124  2934668194  2934668195
  2934668196  2934668197  2934668198  2934668199  7016368434  2934668200
 11405918336  2934670002 11434275747 10882554975  2934670004  2934670005
  2934670006  2901394135  2901394134  4440166373]

*** Tags:
	 ['k="admin_level" v="6"', 'k="boundary" v="administrative"']


In [102]:
k = 0 

line_coordinates = [] 

for r in wn:
    k = k + 1 
    xlat = get_values_from_ref( node_lat, node_ref, r ) 
    xlon = get_values_from_ref( node_lon, node_ref, r )  
    print( "Nodes No. k, ref, lat, lon =", k, xlat, xlon )
    line_coordinates = np.append ( line_coordinates, np.array( [ xlat, xlon ] ) )
#

Nodes No. k, ref, lat, lon = 1 10.8002412 106.667571
Nodes No. k, ref, lat, lon = 2 10.8003054 106.6674561
Nodes No. k, ref, lat, lon = 3 10.8005947 106.6670142
Nodes No. k, ref, lat, lon = 4 10.8008714 106.6675908
Nodes No. k, ref, lat, lon = 5 10.8010031 106.6677625
Nodes No. k, ref, lat, lon = 6 10.8011614 106.6678129
Nodes No. k, ref, lat, lon = 7 10.8011939 106.6678233
Nodes No. k, ref, lat, lon = 8 10.8016289 106.6679798
Nodes No. k, ref, lat, lon = 9 10.8021071 106.6681219
Nodes No. k, ref, lat, lon = 10 10.8022731 106.6681343
Nodes No. k, ref, lat, lon = 11 10.802311 106.6681371
Nodes No. k, ref, lat, lon = 12 10.8024502 106.6681031
Nodes No. k, ref, lat, lon = 13 10.8026083 106.6680274
Nodes No. k, ref, lat, lon = 14 10.802906 106.6682929
Nodes No. k, ref, lat, lon = 15 10.8030773 106.668537
Nodes No. k, ref, lat, lon = 16 10.8031089 106.6688106
Nodes No. k, ref, lat, lon = 17 10.8030539 106.6689713
Nodes No. k, ref, lat, lon = 18 10.8043923 106.6692396
Nodes No. k, ref, lat, 

In [103]:
line_coordinates

array([  10.8002412,  106.667571 ,   10.8003054,  106.6674561,
         10.8005947,  106.6670142,   10.8008714,  106.6675908,
         10.8010031,  106.6677625,   10.8011614,  106.6678129,
         10.8011939,  106.6678233,   10.8016289,  106.6679798,
         10.8021071,  106.6681219,   10.8022731,  106.6681343,
         10.802311 ,  106.6681371,   10.8024502,  106.6681031,
         10.8026083,  106.6680274,   10.802906 ,  106.6682929,
         10.8030773,  106.668537 ,   10.8031089,  106.6688106,
         10.8030539,  106.6689713,   10.8043923,  106.6692396,
         10.8066259,  106.6694714,   10.8068623,  106.6699244,
         10.8076717,  106.6700132,   10.807599 ,  106.6706626,
         10.8078567,  106.671015 ,   10.8078704,  106.6710503,
         10.8086673,  106.6710643,   10.808717 ,  106.6706553,
         10.8099331,  106.6706833,   10.8100633,  106.6715986,
         10.8111621,  106.672252 ,   10.8117951,  106.6721306,
         10.8122354,  106.6720652,   10.8130494,  106.6

In [104]:
line_coordinates = line_coordinates.reshape( ( -1, 2) )
line_coordinates

array([[  10.8002412,  106.667571 ],
       [  10.8003054,  106.6674561],
       [  10.8005947,  106.6670142],
       [  10.8008714,  106.6675908],
       [  10.8010031,  106.6677625],
       [  10.8011614,  106.6678129],
       [  10.8011939,  106.6678233],
       [  10.8016289,  106.6679798],
       [  10.8021071,  106.6681219],
       [  10.8022731,  106.6681343],
       [  10.802311 ,  106.6681371],
       [  10.8024502,  106.6681031],
       [  10.8026083,  106.6680274],
       [  10.802906 ,  106.6682929],
       [  10.8030773,  106.668537 ],
       [  10.8031089,  106.6688106],
       [  10.8030539,  106.6689713],
       [  10.8043923,  106.6692396],
       [  10.8066259,  106.6694714],
       [  10.8068623,  106.6699244],
       [  10.8076717,  106.6700132],
       [  10.807599 ,  106.6706626],
       [  10.8078567,  106.671015 ],
       [  10.8078704,  106.6710503],
       [  10.8086673,  106.6710643],
       [  10.808717 ,  106.6706553],
       [  10.8099331,  106.6706833],
 

In [105]:
# Then,


qref = qn[0]

qlat = get_values_from_ref( node_lat, node_ref, qref ) 
qlon = get_values_from_ref( node_lon, node_ref, qref )  

map_center = [ qlat, qlon ]

print ( 'map center point = ( %15.10f,  %15.10f ), node reference = %15d ' % ( qlat, qlon, qref ) ) 

#
#

map_osm = folium.Map ( location=map_center, zoom_start=13 )

folium.Marker(
    location=map_center,
    tooltip="Click me!",
    popup="Center point",
    icon=folium.Icon(icon="cloud", color="purple"),
).add_to(map_osm)

folium.PolyLine ( 
    line_coordinates, 
    color="blue", 
    weight=2, 
    opacity=0.7 
).add_to( map_osm )


map_osm  

map center point = (   10.8025830000,   106.6521157000 ), node reference =     11214914766 


In [106]:
list1 = [1,2,3]
list2 = [4,5,6,7,8]

print ( list1 )
print ( list2 )

biglist = []
biglist.append( list(list1) )
biglist.append( list(list2) )

print ( biglist )

print ( len(biglist) )
print ( biglist[0], len(biglist[0]) )
print ( biglist[1], len(biglist[1]) )


[1, 2, 3]
[4, 5, 6, 7, 8]
[[1, 2, 3], [4, 5, 6, 7, 8]]
2
[1, 2, 3] 3
[4, 5, 6, 7, 8] 5


In [107]:
qw_lines = [] 

for j in range( len(qw)):
    
    wref = qw [ j ]
    wval = wayinfo_from_ref ( wref )    
    
    print ('\n*** Way: ', j+1, wref )
    
    wn, wt = way_has_refs_n_tags ( wval ) 
    
    print ( 'Node:', wn ) 
    print ( 'Tags:', wt ) 

    k = 0 
    line_coordinates = [] 
    for r in wn:
        k = k + 1 
        xlat = get_values_from_ref( node_lat, node_ref, r ) 
        xlon = get_values_from_ref( node_lon, node_ref, r )  
        print( "\t Nodes No. k, ref, lat, lon =", k, xlat, xlon )
        line_coordinates = np.append ( line_coordinates, np.array( [ xlat, xlon ] ) )
#    
    qw_lines.append( list(line_coordinates) ) 
#
    


*** Way:  1 289931217
Node: [ 4191172081  2322648184  4191172094  4191172103  4191172113  5806611596
  2934668192  4191172118  4191172119  8414354136  2934668193  4191172120
  4191172121  4191172122  4191172123  4191172124  2934668194  2934668195
  2934668196  2934668197  2934668198  2934668199  7016368434  2934668200
 11405918336  2934670002 11434275747 10882554975  2934670004  2934670005
  2934670006  2901394135  2901394134  4440166373]
Tags: ['k="admin_level" v="6"', 'k="boundary" v="administrative"']
	 Nodes No. k, ref, lat, lon = 1 10.8002412 106.667571
	 Nodes No. k, ref, lat, lon = 2 10.8003054 106.6674561
	 Nodes No. k, ref, lat, lon = 3 10.8005947 106.6670142
	 Nodes No. k, ref, lat, lon = 4 10.8008714 106.6675908
	 Nodes No. k, ref, lat, lon = 5 10.8010031 106.6677625
	 Nodes No. k, ref, lat, lon = 6 10.8011614 106.6678129
	 Nodes No. k, ref, lat, lon = 7 10.8011939 106.6678233
	 Nodes No. k, ref, lat, lon = 8 10.8016289 106.6679798
	 Nodes No. k, ref, lat, lon = 9 10.802107

	 Nodes No. k, ref, lat, lon = 15 10.8127061 106.633054
	 Nodes No. k, ref, lat, lon = 16 10.8119034 106.6332945
	 Nodes No. k, ref, lat, lon = 17 10.8107084 106.6336588
	 Nodes No. k, ref, lat, lon = 18 10.8094447 106.6340464
	 Nodes No. k, ref, lat, lon = 19 10.8089361 106.6342127
	 Nodes No. k, ref, lat, lon = 20 10.8083667 106.634399
	 Nodes No. k, ref, lat, lon = 21 10.8080066 106.6345297
	 Nodes No. k, ref, lat, lon = 22 10.8069232 106.6348174
	 Nodes No. k, ref, lat, lon = 23 10.8066717 106.6348987
	 Nodes No. k, ref, lat, lon = 24 10.8063048 106.635017
	 Nodes No. k, ref, lat, lon = 25 10.8062997 106.6350187
	 Nodes No. k, ref, lat, lon = 26 10.8062723 106.6350275
	 Nodes No. k, ref, lat, lon = 27 10.8059318 106.6351372
	 Nodes No. k, ref, lat, lon = 28 10.8038388 106.6357729

*** Way:  13 466939789
Node: [ 4102230923  4616320890  4616320891 11466006094  4616320892 11466006093
  4616320894  6726987255  4616320895  4616320896  4616320897  5772194297
  4616320898  4616320899  461

	 Nodes No. k, ref, lat, lon = 17 10.775944 106.6534516
	 Nodes No. k, ref, lat, lon = 18 10.7761028 106.6536022
	 Nodes No. k, ref, lat, lon = 19 10.7763945 106.6538934
	 Nodes No. k, ref, lat, lon = 20 10.7780951 106.6555345
	 Nodes No. k, ref, lat, lon = 21 10.7785007 106.655893

*** Way:  22 468652914
Node: [4617816373 4617816374 5717194969]
Tags: ['k="admin_level" v="6"', 'k="boundary" v="administrative"']
	 Nodes No. k, ref, lat, lon = 1 10.8329889 106.6323471
	 Nodes No. k, ref, lat, lon = 2 10.8328638 106.6329063
	 Nodes No. k, ref, lat, lon = 3 10.8323551 106.6339409

*** Way:  23 468659632
Node: [4905120025 8408128760 4905120024 2074325728 2074325427 4617816403
 4617816404 4617816405 4617816406 4617816407 4617816408 4617816409
 4617816410 4617816411 4617816412 4617816413 4617816414 4617816415
 4617816416 4617816417 4617816418 9338769566  366402186 4617816420]
Tags: ['k="admin_level" v="6"', 'k="boundary" v="administrative"']
	 Nodes No. k, ref, lat, lon = 1 10.826905 106.6778

	 Nodes No. k, ref, lat, lon = 5 10.7887238 106.6652123
	 Nodes No. k, ref, lat, lon = 6 10.7890372 106.6651347
	 Nodes No. k, ref, lat, lon = 7 10.7891739 106.6651008
	 Nodes No. k, ref, lat, lon = 8 10.7901154 106.6647957
	 Nodes No. k, ref, lat, lon = 9 10.7901552 106.6650503
	 Nodes No. k, ref, lat, lon = 10 10.7897021 106.6657263
	 Nodes No. k, ref, lat, lon = 11 10.7893609 106.6661373
	 Nodes No. k, ref, lat, lon = 12 10.7893372 106.6662339
	 Nodes No. k, ref, lat, lon = 13 10.7894645 106.6662742
	 Nodes No. k, ref, lat, lon = 14 10.7895641 106.6667537

*** Way:  30 496065784
Node: [ 6258204836 11897986026 11897986025  8261270562 11897986024 11897986023
 11897986022  4879063679  8262886820 11897986021  8262886825 11897986020
 11897986019 11897986018  8262886826  8262886827  8262886828  8262886829
 11863150557  8262886831  8262886830]
Tags: ['k="admin_level" v="6"', 'k="boundary" v="administrative"']
	 Nodes No. k, ref, lat, lon = 1 10.7895641 106.6667537
	 Nodes No. k, ref, lat, 

In [108]:
for n in range ( len(qw_lines) ):

    #qw_lines[n] = np.array( qw_lines[n] ).reshape( ( -1, 2) )
    
    print ( n+1, qw_lines[n], len(qw_lines[n]) )
    
    print ( np.array( qw_lines[n] ).reshape( ( -1, 2) ) )
    

1 [10.8002412, 106.667571, 10.800305399999999, 106.6674561, 10.8005947, 106.6670142, 10.8008714, 106.6675908, 10.801003100000001, 106.66776249999999, 10.8011614, 106.6678129, 10.801193899999999, 106.66782329999999, 10.801628900000001, 106.6679798, 10.802107100000001, 106.6681219, 10.802273100000001, 106.66813430000001, 10.802311, 106.6681371, 10.802450199999999, 106.6681031, 10.802608299999999, 106.6680274, 10.802906, 106.6682929, 10.8030773, 106.668537, 10.8031089, 106.6688106, 10.8030539, 106.6689713, 10.8043923, 106.6692396, 10.8066259, 106.66947140000001, 10.806862300000001, 106.6699244, 10.8076717, 106.6700132, 10.807599, 106.6706626, 10.8078567, 106.671015, 10.807870400000001, 106.6710503, 10.8086673, 106.6710643, 10.808717, 106.67065530000001, 10.8099331, 106.67068329999999, 10.810063299999999, 106.6715986, 10.811162100000001, 106.672252, 10.811795099999999, 106.6721306, 10.8122354, 106.67206520000001, 10.813049400000001, 106.67213409999999, 10.813812, 106.67239600000001, 10.814

In [109]:
#
#

map_osm = folium.Map ( location=map_center, zoom_start=13 )

folium.Marker(
    location=map_center,
    tooltip="Click me!",
    popup="Center point",
    icon=folium.Icon(icon="cloud", color="purple"),
).add_to(map_osm)


for n in range ( len(qw_lines) ):
    lines =  np.array( qw_lines[n] ).reshape( ( -1, 2) )    
    folium.PolyLine ( 
        lines, 
        color="blue", 
        weight=2, 
        opacity=0.7 
    ).add_to( map_osm )
#

# Save as HTML:
#map_osm.save ( "get_OSM_Quan5_boundary.html" )


map_osm  

# How to make that boundary to be closed ?

In [110]:
flags = np.zeros( len(qw_lines) )
print ( flags )
print ( np.count_nonzero(flags) )

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0


In [111]:
fruits = ["apple", "banana", "cherry"]
for x in fruits:
    if x == "banana":
        continue
    print(x)

apple
cherry


In [112]:
flags = np.zeros( len(qw_lines) )
print ( flags )
print ( np.count_nonzero(flags) )

n = 0 
blines = np.copy( np.array( qw_lines[n] ).reshape( ( -1, 2) )  ) 
btail  = blines[-1]

print ( blines )

#print ( blines[::-1] )

print ( 'btail:', btail )

eps = 1.0e-2

nl = len(qw_lines)
flags[0] = 1 

while ( np.count_nonzero(flags) < nl ):
#
    ih = 0
    hmin = 1.0e+10 
    it = 0 
    tmin = 1.0e+10 
#    
    for n in range ( 1, len(qw_lines) ):
#        
        if ( flags[n] > 0 ):
            continue
#            
        xlines =  np.copy( np.array( qw_lines[n] ).reshape( ( -1, 2) ) ) 
        xhead = xlines[0]
        xtail = xlines[-1]        
        h1 = np.max( np.abs(xhead - btail) ) 
        t1 = np.max( np.abs(xtail - btail) )                         
#        
        if ( h1 < hmin ):
            ih = n
            hmin = h1             
#
        if ( t1 < tmin ): 
            it = n 
            tmin = t1 
#    
    if ( hmin < tmin ):
        n = ih 
        xlines =  np.copy( np.array( qw_lines[n] ).reshape( ( -1, 2) ) ) 
        xhead = xlines[0]
        xtail = xlines[-1]            
        print( n, "Ok. btail -- xhead, xhead =", xhead )
        blines = np.append( blines, xlines, axis=0 )
        btail  = blines[-1]
        print ( 'xtail:', xtail )        
        print ( 'btail:', btail )     
        flags[n] = 1 
    else:
        n = it
        xlines =  np.copy( np.array( qw_lines[n] ).reshape( ( -1, 2) ) ) 
        xhead = xlines[0]
        xtail = xlines[-1]                    
        print( n, "Ok. btail -- xtail, xtail =", xtail )
        blines = np.append( blines, xlines[::-1], axis=0 )
        btail  = blines[-1]
        print ( 'xhead:', xhead )
        print ( 'btail:', btail )
        flags[n] = 1        
#
# To make the polyline to be closed, i.e., becomes a polygon:

blines = np.append( blines, blines[0] ).reshape( ( -1, 2) )

print ( blines )

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
0
[[  10.8002412  106.667571 ]
 [  10.8003054  106.6674561]
 [  10.8005947  106.6670142]
 [  10.8008714  106.6675908]
 [  10.8010031  106.6677625]
 [  10.8011614  106.6678129]
 [  10.8011939  106.6678233]
 [  10.8016289  106.6679798]
 [  10.8021071  106.6681219]
 [  10.8022731  106.6681343]
 [  10.802311   106.6681371]
 [  10.8024502  106.6681031]
 [  10.8026083  106.6680274]
 [  10.802906   106.6682929]
 [  10.8030773  106.668537 ]
 [  10.8031089  106.6688106]
 [  10.8030539  106.6689713]
 [  10.8043923  106.6692396]
 [  10.8066259  106.6694714]
 [  10.8068623  106.6699244]
 [  10.8076717  106.6700132]
 [  10.807599   106.6706626]
 [  10.8078567  106.671015 ]
 [  10.8078704  106.6710503]
 [  10.8086673  106.6710643]
 [  10.808717   106.6706553]
 [  10.8099331  106.6706833]
 [  10.8100633  106.6715986]
 [  10.8111621  106.672252 ]
 [  10.8117951  106.6721306]
 

In [113]:
print ( blines )
print ( blines[0] )
print ( blines[-1] )

[[  10.8002412  106.667571 ]
 [  10.8003054  106.6674561]
 [  10.8005947  106.6670142]
 [  10.8008714  106.6675908]
 [  10.8010031  106.6677625]
 [  10.8011614  106.6678129]
 [  10.8011939  106.6678233]
 [  10.8016289  106.6679798]
 [  10.8021071  106.6681219]
 [  10.8022731  106.6681343]
 [  10.802311   106.6681371]
 [  10.8024502  106.6681031]
 [  10.8026083  106.6680274]
 [  10.802906   106.6682929]
 [  10.8030773  106.668537 ]
 [  10.8031089  106.6688106]
 [  10.8030539  106.6689713]
 [  10.8043923  106.6692396]
 [  10.8066259  106.6694714]
 [  10.8068623  106.6699244]
 [  10.8076717  106.6700132]
 [  10.807599   106.6706626]
 [  10.8078567  106.671015 ]
 [  10.8078704  106.6710503]
 [  10.8086673  106.6710643]
 [  10.808717   106.6706553]
 [  10.8099331  106.6706833]
 [  10.8100633  106.6715986]
 [  10.8111621  106.672252 ]
 [  10.8117951  106.6721306]
 [  10.8122354  106.6720652]
 [  10.8130494  106.6721341]
 [  10.813812   106.672396 ]
 [  10.8141438  106.6727813]
 [  10.8141438

In [114]:
#
#

map_osm = folium.Map ( location=map_center, zoom_start=13 )

folium.Marker(
    location=map_center,
    tooltip="Click me!",
    popup="Center point",
    icon=folium.Icon(icon="cloud", color="purple"),
).add_to(map_osm)


folium.Polygon (
    blines,
    weight=1,
    color="red",         # Border color
    fill=True,            # Fill the polygon
    fill_color="cyan",  # Fill color
    fill_opacity=0.2      # Opacity of the fill
).add_to ( map_osm )


# Save as HTML:
#map_osm.save ( "get_OSM_QuanTanBinh_boundary.html" )


map_osm  

# Now we combine all stuffs for RELATION

In [115]:
def relation_has_tags ( rval ):
    tags = []
    for x in re.findall( r'<tag\b(.*?)\/>', rval ):
        tags.append( trim(x) )
    return tags 
#

def relation_has_members ( rval ):
    members = []
    for x in re.findall( r'<member\b(.*?)\/>', rval ):
        members.append( trim(x) )
    return members 
#

def relation_has_member_ref ( rval, type='node' ):
#    
#   type='node'
#   type='way'
#   type='relation'
#
    def get_ref ( ele ):
        k  = re.sub( r'^.*\s+ref="([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)"\s+.*$', r'\1', ele )
        return np.dtype('int64').type( k )
#    
    t = str( 'type="' + str(type) + '"' ) 
    m = relation_has_members ( rval )
    r = []
    for k in range ( len(m) ):
        if ( re.split( ' ', m[k] )[0] == t ) :
            j = get_ref ( m[k] )
            r.append(j)
    return r 
#

def relation_has_ref_of_way ( rval ):    
    return sorted(relation_has_member_ref ( rval, type="way" ))
#

def relation_has_ref_of_node ( rval ):
    return sorted(relation_has_member_ref ( rval, type='node' ))
#    

def relation_has_ref_of_relation ( rval ):
    return sorted(relation_has_member_ref ( rval, type='relation' ))
#    

def relation_has_refs_n_tags ( rval ):
    return \
    relation_has_ref_of_node (rval), \
    relation_has_ref_of_way ( rval ), \
    relation_has_ref_of_relation ( rval ), \
    relation_has_tags ( rval )
#



In [116]:
relation_has_refs_n_tags ( tphcm )

([369486734],
 [118908344,
  118908598,
  140643239,
  140674265,
  205798558,
  205798559,
  205958089,
  291052401,
  291501521,
  291549078,
  291966910,
  300768010,
  369220580,
  369220609,
  369220612,
  470198972,
  470198973,
  471507118,
  485048429,
  485048430,
  485524300,
  485524305,
  485529720,
  486392495,
  486427207,
  486434217,
  486546209,
  486553007,
  486715823,
  486715824,
  486715826,
  486718195,
  486718196,
  486718197,
  501631548,
  501631554,
  504562909,
  504562911,
  514726081,
  514726082,
  514726103,
  514726105,
  515126243,
  515126246,
  515132520,
  529412947,
  529412949,
  529412955,
  529412968,
  530029777,
  530029783,
  530029792,
  530029793,
  530033550,
  530033555,
  530221399,
  531200831,
  532477271,
  532477299,
  532477302,
  567825872,
  567825873,
  606263395,
  606263411,
  606280131,
  606600077,
  606600078,
  606609439,
  607618296,
  607618302,
  607618311,
  607905791,
  608005582,
  608005584,
  684805581,
  684805589

In [136]:
print ( tphcm )

<member type="node" ref="369486734" role="admin_centre"/><member type="relation" ref="2587287" role="subarea"/><member type="relation" ref="3819816" role="subarea"/><member type="relation" ref="2778323" role="subarea"/><member type="relation" ref="3820432" role="subarea"/><member type="relation" ref="6228792" role="subarea"/><member type="relation" ref="2764875" role="subarea"/><member type="relation" ref="6888445" role="subarea"/><member type="relation" ref="6228121" role="subarea"/><member type="relation" ref="6846181" role="subarea"/><member type="relation" ref="6923167" role="subarea"/><member type="relation" ref="7157268" role="subarea"/><member type="relation" ref="3797166" role="subarea"/><member type="relation" ref="7157255" role="subarea"/><member type="relation" ref="7157220" role="subarea"/><member type="relation" ref="7157219" role="subarea"/><member type="relation" ref="7157197" role="subarea"/><member type="relation" ref="3851694" role="subarea"/><member type="relation" r

In [143]:
tpn, tpw, tpr, tpt = relation_has_refs_n_tags ( tphcm ) 

print ( '\n*** Relation value:\n\t', tphcm ) 
print ( '\n*** Node ref:\n\t', tpn ) 
print ( '\n*** Way ref:\n\t', tpw ) 
print ( '\n*** Relation ref:\n\t', tpr ) 
print ( '\n*** Tags:\n\t', tpt ) 


*** Relation value:
	 <member type="node" ref="369486734" role="admin_centre"/><member type="relation" ref="2587287" role="subarea"/><member type="relation" ref="3819816" role="subarea"/><member type="relation" ref="2778323" role="subarea"/><member type="relation" ref="3820432" role="subarea"/><member type="relation" ref="6228792" role="subarea"/><member type="relation" ref="2764875" role="subarea"/><member type="relation" ref="6888445" role="subarea"/><member type="relation" ref="6228121" role="subarea"/><member type="relation" ref="6846181" role="subarea"/><member type="relation" ref="6923167" role="subarea"/><member type="relation" ref="7157268" role="subarea"/><member type="relation" ref="3797166" role="subarea"/><member type="relation" ref="7157255" role="subarea"/><member type="relation" ref="7157220" role="subarea"/><member type="relation" ref="7157219" role="subarea"/><member type="relation" ref="7157197" role="subarea"/><member type="relation" ref="3851694" role="subarea"/><m

In [158]:
qref = tpn[0]

qlat = get_values_from_ref( node_lat, node_ref, qref ) 
qlon = get_values_from_ref( node_lon, node_ref, qref )  

map_center = [ qlat, qlon ]

print ( 'map center point = ( %15.10f,  %15.10f ), node reference = %15d ' % ( qlat, qlon, qref ) ) 


map center point = (   10.7763897000,   106.7011391000 ), node reference =       369486734 


In [159]:
def relation_get_boundary ( tpw, report=False ):

    tpw_lines = [] 

    for j in range( len(tpw) ):

        wref = tpw [ j ]
        wval = wayinfo_from_ref ( wref )    
        
        if ( report ):
            print ('\n*** Way: ', j+1, wref )

        wn, wt = way_has_refs_n_tags ( wval ) 

        if ( report ):
            print ( 'Node:', wn ) 
            print ( 'Tags:', wt ) 

        k = 0 
        line_coordinates = [] 
        for r in wn:
            k = k + 1 
            xlat = get_values_from_ref( node_lat, node_ref, r ) 
            xlon = get_values_from_ref( node_lon, node_ref, r )  
            if ( report ):
                print( "\t Nodes No. k, ref, lat, lon =", k, xlat, xlon )
            line_coordinates = np.append ( line_coordinates, np.array( [ xlat, xlon ] ) )
    #    
        tpw_lines.append( list(line_coordinates) ) 
    #


    flags = np.zeros( len(tpw_lines) )
    
    if ( report ):
        print ( flags )
        print ( np.count_nonzero(flags) )

    n = 0 
    blines = np.copy( np.array( tpw_lines[n] ).reshape( ( -1, 2) )  ) 
    btail  = blines[-1]

    if ( report ):    
        print ( blines )
        print ( 'btail:', btail )

    eps = 1.0e-2

    nl = len(tpw_lines)
    flags[0] = 1 

    while ( np.count_nonzero(flags) < nl ):
    #
        ih = 0
        hmin = 1.0e+10 
        it = 0 
        tmin = 1.0e+10 
    #    
        for n in range ( 1, len(tpw_lines) ):
    #        
            if ( flags[n] > 0 ):
                continue
    #            
            xlines =  np.copy( np.array( tpw_lines[n] ).reshape( ( -1, 2) ) ) 
            xhead = xlines[0]
            xtail = xlines[-1]        
            h1 = np.max( np.abs(xhead - btail) ) 
            t1 = np.max( np.abs(xtail - btail) )                         
    #        
            if ( h1 < hmin ):
                ih = n
                hmin = h1             
    #
            if ( t1 < tmin ): 
                it = n 
                tmin = t1 
    #    
        if ( hmin < tmin ):
            n = ih 
            xlines =  np.copy( np.array( tpw_lines[n] ).reshape( ( -1, 2) ) ) 
            xhead = xlines[0]
            xtail = xlines[-1]       
            if ( report ):            
                print( n, "Ok. btail -- xhead, xhead =", xhead )
            blines = np.append( blines, xlines, axis=0 )
            btail  = blines[-1]
            if ( report ):            
                print ( 'xtail:', xtail )        
                print ( 'btail:', btail )     
            flags[n] = 1 
        else:
            n = it
            xlines =  np.copy( np.array( tpw_lines[n] ).reshape( ( -1, 2) ) ) 
            xhead = xlines[0]
            xtail = xlines[-1]        
            if ( report ):            
                print( n, "Ok. btail -- xtail, xtail =", xtail )
            blines = np.append( blines, xlines[::-1], axis=0 )
            btail  = blines[-1]
            if ( report ):            
                print ( 'xhead:', xhead )
                print ( 'btail:', btail )
            flags[n] = 1        
    #
    # To make the polyline to be closed, i.e., becomes a polygon:

    blines = np.append( blines, blines[0] ).reshape( ( -1, 2) )

    return  blines

In [168]:
blines = relation_get_boundary ( tpw )

In [169]:
#
#

map_osm = folium.Map ( location=map_center, zoom_start=10 )

folium.Marker(
    location=map_center,
    tooltip="Click me!",
    popup="Center point",
    icon=folium.Icon(icon="cloud", color="purple"),
).add_to(map_osm)


folium.Polygon (
    blines,
    weight=1,
    color="blue",         # Border color
    fill=True,            # Fill the polygon
    fill_color="cyan",  # Fill color
    fill_opacity=0.2      # Opacity of the fill
).add_to ( map_osm )


# Save as HTML:
map_osm.save ( "get_OSM_tpHCM_boundary.html" )


map_osm  

In [162]:
print ( quan_lst, len(quan_lst) ) 
print ( quan_ref ) 

['Quận 1', 'Quận 7', 'Quận 4', 'Quận Bình Thạnh', 'Quận 3', 'Quận 5', 'Quận Phú Nhuận', 'Quận 10', 'Quận 6', 'Quận Tân Phú', 'Quận Tân Bình', 'Quận 11', 'Quận 8', 'Quận Gò Vấp', 'Quận Bình Tân', 'Quận 12'] 16
[47, 57, 68, 111, 134, 135, 157, 260, 267, 281, 293, 295, 312, 351, 368, 379]


In [163]:
k =  0 
rref = quan_ref [ k ] 
#rval = get_inform_from_ref( rel_info, rel_ref, rref )
rval = rel_info [ rref ]
print ( rval )

<member type="way" ref="287982010" role="outer"/><member type="way" ref="189393381" role="outer"/><member type="way" ref="189618477" role="outer"/><member type="way" ref="418298179" role="outer"/><member type="way" ref="418298182" role="outer"/><member type="way" ref="189233380" role="outer"/><member type="way" ref="192038542" role="outer"/><member type="way" ref="192680136" role="outer"/><member type="way" ref="189619670" role="outer"/><member type="way" ref="206677421" role="outer"/><member type="way" ref="472074786" role="outer"/><member type="way" ref="472074778" role="outer"/><member type="way" ref="191877594" role="outer"/><member type="way" ref="472236749" role="outer"/><member type="way" ref="191877592" role="outer"/><member type="way" ref="286342012" role="outer"/><member type="way" ref="888760606" role="outer"/><member type="way" ref="286341995" role="outer"/><member type="way" ref="191881069" role="outer"/><member type="way" ref="289931230" role="outer"/><member type="way" r

In [164]:
import random


hexadecimal_alphabets = '0123456789ABCDEF'

number_of_colors = 30 
color = ["#" + ''.join([random.choice(hexadecimal_alphabets) for j in range(6)]) for i in range(number_of_colors)]
color 

['#A83DA5',
 '#D61E12',
 '#668F91',
 '#BBBB20',
 '#BD1CA6',
 '#58E742',
 '#7A330A',
 '#7A3B5F',
 '#436B1E',
 '#D0EECC',
 '#7A8B4D',
 '#3441AB',
 '#D3680D',
 '#10BEF1',
 '#0FCA44',
 '#926BD0',
 '#BF0BBC',
 '#4BBB12',
 '#C9D762',
 '#31349E',
 '#887A85',
 '#88EA93',
 '#4FB8EB',
 '#F03BBD',
 '#78F1DC',
 '#834F44',
 '#AC9E53',
 '#DE6FC0',
 '#0DACC3',
 '#2755BF']

In [171]:
map_osm = folium.Map ( location=map_center, zoom_start=11 )

folium.Marker(
    location=map_center,
    tooltip="Click me!",
    popup="Center point",
    icon=folium.Icon(icon="cloud", color="purple"),
).add_to(map_osm)


for k in range( len(quan_ref) ) : 
    rref = quan_ref [ k ]    
    rval = rel_info [ rref ]

    #print( rval ) 
    
    rn, rw, rr, rt = relation_has_refs_n_tags ( rval ) 
    #print ( '\n*** Node ref:\n\t', rn ) 
    #print ( '\n*** Way ref:\n\t', rw ) 
    #print ( '\n*** Relation ref:\n\t', rr ) 
    #print ( '\n*** Tags:\n\t', rt ) 
    
    blines = relation_get_boundary ( rw )
    
    folium.Polygon (
    blines,
    weight=1,
    color="blue",         # Border color
    fill=True,            # Fill the polygon
    fill_color=color[k],  # Fill color
    fill_opacity=0.2      # Opacity of the fill
    ).add_to ( map_osm )
#

#map_osm.save ( "get_OSM_tpHCM_Quan.html" )

map_osm